Code for Isotope Project Fall 2024 made by Daniel Boupha Christensen

In [ ]:
# import all libraries needed for this project
# Run this cell
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
import math
import re
from google.colab import files
from google.colab import drive
from matplotlib.backends.backend_pdf import PdfPages
drive.mount('/content/drive')

You can either mount the CSV files from google drive and get the file path from there or get the file path from your local drive.

In [ ]:
# Data wrangling, manipulation, and visualization methods.
# Run this cell and do not edit it unless it is specified within a function that you can

#Global Variables
"""These dictionaries will be have the same keys in the following format of keys and values:
{
  'Isolation': {
    'Isotope': {
      'Parent': {
        'Daughter': {
          'Normalized Collision Energy Value': <value>
        }
      },
      'Daughter': {
        'Parent': {
          'Normalized Collision Energy Value': <value>
        }
      }
    }
  }
}
"""
All_alphas_dict_PD = {} # Gets values from the all_calculations_and_graphs function. It will be updated everytime you use the function
All_enrichments_dict_PD= {} # Gets values from the all_calculations_and_graphs function. It will be updated everytime you use the function
All_Errors_dict_PD = {} # Gets values from the all_calculations_and_graphs function. It will be updated everytime you use the function
All_Parent_Areas_dict_PD = {} # Gets values from the all_calculations_and_graphs function. It will be updated everytime you use the function
All_Daughter_Areas_dict_PD = {} #Gets values from the all_calculations_and_graphs function. It will be updated everytime you use the function.
Daughter_NCE_Values_PD = [] # Gets values from the all_calculations_and_graphs function. It will be updated everytime you use the function
Parent_NCE_Values_PD = [] # Gets values from the all_calculations_and_graphs function. It will be updated everytime you use the function

"""All These dictionaries will have the same keys which are the dates the data was taken"""
All_alphas = {} # Gets values in the Error class in the calculate_δα_indv function. It will be updated everytime you use the function
All_enrichments = {} # Gets values in the Error class in the calculate_δα_indv function. It will be updated everytime you use the function
All_Errors = {} # Gets values in the Error class in the calculate_δα_indv function. It will be updated everytime you use the function
All_Parent_Areas = {} # Gets values in the Error class in the calculate_δα_indv function. It will be updated everytime you use the function
All_Daughter_Areas = {} # Gets values in the Error class in the calculate_δα_indv function. It will be updated everytime you use the function
All_Isotopes = () # Gets values in the Error class in the calculate_δα_indv function.This occurs once as all the files should contain the same isotopes.
Parent_masses = () # Gets values in the Error class in the calculate_δα_indv function. This only occurs once as all the files should contain the same parent masses.
Daughter_masses = () # Gets values in the Error class in the calculate_δα_indv function. This only occurs once as all the files should contain the same daughter masses.
Atom_Names = " " # Gets values in the Error class in the calculate_δα_indv function. It will be updated once, since this code doesn't function for graphing different isotopes of different molecules such as with Dy and Nd molecules

#Global Functions
def clear_GV_PD():
  """Clears all global variables except the one's that require the use and clearing of the global varibales in clearGV function. They are labeled _PD after the variable name since the distinction between
  Parent NCE and Daughter needs to be made.This is for the graphs data analysis that wants to compare individual aspects of the data from multiple isolations and requires the analysis of all
  data to be done first, so you use this function in conjunction with clearGV if you want to completely start over."""
  global All_alphas_dict_PD
  global All_enrichments_dict_PD
  global All_Errors_dict_PD
  global All_Parent_Areas_dict_PD
  global All_Daughter_Areas_dict_PD
  global Daughter_NCE_Values_PD
  global Parent_NCE_Values_PD

  All_alphas_dict_PD = {}
  All_enrichments_dict_PD= {}
  All_Errors_dict_PD = {}
  All_Parent_Areas_dict_PD = {}
  All_Daughter_Areas_dict_PD = {}
  Daughter_NCE_Values_PD = []
  Parent_NCE_Values_PD = []

def clearGV():
  """Clears global variables not relating to the global variables in ."""
  global All_alphas
  global All_enrichments
  global All_Errors
  global All_Parent_Areas
  global All_Daughter_Areas
  global All_Isotopes
  global Parent_masses
  global Daughter_masses
  global Atom_Names
  All_alphas = {}
  All_enrichments = {}
  All_Errors = {}
  All_Parent_Areas = {}
  All_Daughter_Areas = {}
  All_Isotopes = ()
  Parent_masses = ()
  Daughter_masses = ()
  Atom_Names = " "

def Get_chemical_formula_in_Latex(chemical_formula):
    """
    Takes a chemical formula as input and returns the LaTeX formatted string.
    Handles subscripts, superscripts, and charges.
    """
    # Step 1: Handle groups with subscripts (e.g., NO3 -> NO_3, SO4 -> SO_4)
    formula = re.sub(r'([A-Za-z]+)(\d+)', r'\1_\2', chemical_formula)  # For elements with subscripts like NaCl2
    formula = re.sub(r'(NO3|SO4|CO3|PO4|Cl|Br|I|NH4)(\d*)', r'\1_\2', formula)  # Specific group formatting

    # Step 2: Handle superscript for charges (e.g., - -> ^{-})
    formula = re.sub(r'\^-(?=\])', r'^{-}', formula)  # For negative charge
    formula = re.sub(r'\^(\d+)', r'^{\1}', formula)  # For positive charge, e.g., Na^+ to Na^{+}

    # Step 3: Wrap the formula with \mathrm{} for proper chemical notation
    latex_formula = r'$\mathrm{' + formula + r'}$'

    return latex_formula

def format_chemical_formula(chemical_formula):
    """
    Takes a LaTeX-style chemical formula with _ and ^ for subscripts and superscripts
    and returns the properly formatted formula as a string.

    Input example:
    - 'H_{2}O' for H₂O
    - 'NO_{3}^{-}' for NO₃⁻
    - 'Nd(NO_{3})_{4}^{-}' for [Nd(NO₃)₄]⁻
    """

    # Define the subscript and superscript maps inside the function
    subscript_map = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    superscript_map = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    # Step 1: Convert _{number} to subscripts for individual elements or groups
    formula = re.sub(r'_{(\d+)}', lambda m: m.group(1).translate(subscript_map), chemical_formula)

    # Step 2: Convert ^{number} or ^{-} to superscripts for charges or other exponents
    formula = re.sub(r'\^{(-?\d+)}', lambda m: m.group(1).translate(superscript_map), formula)
    formula = re.sub(r'\^{(-)}', '⁻', formula)  # Handle the negative sign as a superscript

    # Step 3: Remove curly braces from around groups (i.e., NO_{3} -> NO₃)
    formula = re.sub(r'\{|\}', '', formula)

    # Step 4: Ensure the minus sign is in superscript form where needed
    formula = formula.replace('^-', '⁻')  # Handle the special case for superscript minus

    return formula

    """# Test cases with LaTeX-style input formulas
      input_formula_1 = 'H_{2}O'
      formatted_formula_1 = format_chemical_formula(input_formula_1)
      print(formatted_formula_1)  # Expected Output: H₂O

      input_formula_2 = 'NO_{3}^{-}'
      formatted_formula_2 = format_chemical_formula(input_formula_2)
      print(formatted_formula_2)  # Expected Output: NO₃⁻

      input_formula_3 = 'Nd(NO_{3})_{4}^{-}'
      formatted_formula_3 = format_chemical_formula(input_formula_3)
      print(formatted_formula_3)  # Expected Output: Nd(NO₃)₄⁻

      input_formula_4 = '[Nd(NO_{3})_{4}]^{-}'
      formatted_formula_4 = format_chemical_formula(input_formula_4)
      print(formatted_formula_4)  # Expected Output: [Nd(NO₃)₄]⁻"""

def file_number(file_name):
   num_part = file_name.split('_')[1].replace('.csv', '') if '_' in file_name else ''
   return num_part

def import_google_spreadsheet_file(URL):
    """This function takes the string of a URl of a google sheet and stores it into a pandas dataframe, then returns it.
    If you have any files that are in a google spreadsheet, this function allows you to import it in, however you must Change the sharing
    settings to ‘Anyone with the link’ can view, which will make your Google Sheet publicly accessible in a read-only format.

    This function is already implemented into the classes below that'd use this funtion."""

    # Regular expression to match and capture the necessary part of the URL
    general_url_pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'

    # Replace function to construct the new URL for CSV export
    get_spreadsheet_ID_and_sheet_ID = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + \
                                               (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=csv'

    # Replace using regex to construct the export URL
    New_URL = re.sub(general_url_pattern, get_spreadsheet_ID_and_sheet_ID, URL)

    # Read the CSV from the generated URL, handling bad lines and disabling low memory mode
    try:
        df = pd.read_csv(New_URL, low_memory=False, on_bad_lines='skip')  # Disable low memory mode and skip bad lines
        # Optionally, drop rows that are completely empty
        df_cleaned = df.dropna(how='all')
    except pd.errors.ParserError as e:
        print("Error parsing:", e)
        return pd.DataFrame()  # Return an empty DataFrame on error

    return df_cleaned


def αVIsol(Isotope, constant_NCE='P', NCE_Value_Key_Parent=None, NCE_Value_Key_Daughter=None, save_pdf=False):
    """
    Plot Alpha vs. Isolation for a given Isotope and constant NCE (Parent or Daughter).

    Parameters:
        Isotope (str): The isotope key to focus on.
        constant_NCE (str): 'P' for constant Parent, 'D' for constant Daughter.
        NCE_Value_Key_Parent (str or float): The NCE value for the Parent.
        NCE_Value_Key_Daughter (str or float): The NCE value for the Daughter.
        save_pdf (bool): Whether to save the plot as a PDF.
    """
    # Declare global dictionaries
    global All_alphas_dict_PD
    global All_Errors_dict_PD

    x_vals = []  # Isolation keys (top-level keys in the dictionary)
    y_vals = []  # Alpha values
    y_errs = []  # Error bars

    # Iterate over the isolation keys
    for isolation, isotopes in All_alphas_dict_PD.items():
        if Isotope not in isotopes:
          raise ValueError(f"Isotope '{Isotope}' not found in the specified isolation '{isolation}'.")
          #continue  # Skip if the specified Isotope is not found. The isotope value SHOULD always be in it.

        # Process alphas and errors for the specified isotope
        isotope_alphas = All_alphas_dict_PD[isolation][Isotope]
        isotope_errors = All_Errors_dict_PD[isolation][Isotope]

        if constant_NCE == 'P':
            # Parent is constant; gather alphas and errors for Daughters
            for parent, daughters in isotope_alphas.items():
                for daughter, alpha_value in daughters.items():
                    x_vals.append(isolation)
                    y_vals.append(alpha_value)
                    y_errs.append(isotope_errors[parent][daughter])
        elif constant_NCE == 'D':
            # Daughter is constant; gather alphas and errors for Parents
            for daughter, parents in isotope_alphas.items():
                for parent, alpha_value in parents.items():
                    x_vals.append(isolation)
                    y_vals.append(alpha_value)
                    y_errs.append(isotope_errors[daughter][parent])
        else:
            raise ValueError("Invalid constant_NCE value. Must be 'P' or 'D'.")

    # Determine legend label
    constant_label = f"{constant_NCE}_NCE"  # P_NCE or D_NCE
    legend_label = f"N = {Isotope}, {constant_label} = {NCE_Value_Key_Parent if constant_NCE == 'P' else NCE_Value_Key_Daughter}"

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.errorbar(
        x_vals, y_vals, yerr=y_errs, fmt='o', capsize=5, label=legend_label
    )
    plt.xlabel("Isolation")
    plt.ylabel("Alpha Value")
    plt.title("Alpha vs. Isolation")
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)

    custom_labels = []
    for isotope in All_Isotopes:
      isotope_label = f"$\\mathrm{{^{{{isotope}}}{Atom_Names}}}$"
      custom_labels.append(isotope_label)  # Custom labels
    x_axis = All_Isotopes
    plt.xticks(x_axis, custom_labels)

    # Save or display the plot
    if not save_pdf:
      plt.show()

def αVNCE(Isotope, x_axis_NCE, constant_NCE = None, save_pdf = False):
    """Generates an alpha vs normalized collision energy graph where the x_axis NCE is something that is changed and the constant_NCE is a NCE value the data shares.
    If constant_NCE value is equal to None, it will assume alpha is at a constant NCE value and graph the corresponding alpha vs NCE plot. If it is not none, it assumes the
    plotted alphas are not under the same NCE values."""
    return 3

def Combined_Data_Line_Plot_EVI(Normalization_Number = 1, Normalization_Number_is_total_areas = True, save_pdf = False):
    """Makes an Enrichment vs Isotope graph of all combined data after combined the same data stored in the global variables.
    It does need a Normalization number which is an integer of the value you want to normalize the data by, put 1 if none. If you want to normalize by total areas,
    just put the integer 1 into the function, otherwise put Normalization_Number_is_total_areas = False. Put save_pdf = True if you are in the process of adding this chart to a pdf"""

    # Sums values across all the tuples resulting in one tuple of all the summed areas.
    summed_tuple_Parent = tuple(sum(values) for values in zip(*All_Parent_Areas.values()))
    summed_tuple_Daughter = tuple(sum(values) for values in zip(*All_Daughter_Areas.values()))

    # Turns the tuples into an array to get the ratios which normalizes it
    Parent_Ratios = np.array(summed_tuple_Parent) / np.array(summed_tuple_Parent).sum() if Normalization_Number_is_total_areas else np.array(summed_tuple_Parent) / Normalization_Number
    Daughter_Ratios = np.array(summed_tuple_Daughter) / np.array(summed_tuple_Daughter).sum() if Normalization_Number_is_total_areas else np.array(summed_tuple_Daughter) / Normalization_Number

    # Calculating the errors and enrichments
    Enrichments_of_Combined_Data = tuple((Daughter_Ratios/Parent_Ratios) - 1)
    Errors_of_Combined_Data = tuple((Daughter_Ratios/Parent_Ratios) * np.sqrt(
        (np.sqrt(np.array(summed_tuple_Daughter)) / np.array(summed_tuple_Daughter))**2 +
        (np.sqrt(np.array(summed_tuple_Parent)) / np.array(summed_tuple_Parent))**2))

    # Making the graph
    plt.errorbar(All_Isotopes, Enrichments_of_Combined_Data, yerr = Errors_of_Combined_Data, capsize=3, marker='o', linestyle='-', color = (0.6, 0.8, 1.0), ecolor = (0.5,0.0,0.5), markerfacecolor = (1.0,0.5,0.4), markeredgecolor = (0.9,0.7,0.3))
    plt.xlabel('Isotope')
    plt.ylabel('ε')
    plt.title("ε Trends vs. Isotopic Composition for Combined Data")
    plt.grid(True)
    custom_labels = []
    for isotope in All_Isotopes:
      isotope_label = f"$\\mathrm{{^{{{isotope}}}{Atom_Names}}}$"
      custom_labels.append(isotope_label)  # Custom labels
    x_axis = All_Isotopes
    plt.xticks(x_axis, custom_labels)
    if not save_pdf:
      plt.show()

def Combined_Data_Line_Plot_AVI(Normalization_Number = 1, Normalization_Number_is_total_areas = True, save_pdf = False):
    """Makes an Alpha vs Isotope graph after combined the data stored in the global variables.
    It does need a Normalization number which is an integer of the value you want to normalize the data by, put 1 if none. If you want to normalize by total areas,
    just put the integer 1 into the function, otherwise put Normalization_Number_is_total_areas = False. Put save_pdf = True if you are in the process of adding this chart to a pdf"""

    # Sums values across all the tuples resulting in one tuple of all the summed areas.
    summed_tuple_Parent = tuple(sum(values) for values in zip(*All_Parent_Areas.values()))
    summed_tuple_Daughter = tuple(sum(values) for values in zip(*All_Daughter_Areas.values()))

    # Turns the tuples into an array to get the ratios which normalizes it
    Parent_Ratios = np.array(summed_tuple_Parent) / np.array(summed_tuple_Parent).sum() if Normalization_Number_is_total_areas else np.array(summed_tuple_Parent) / Normalization_Number
    Daughter_Ratios = np.array(summed_tuple_Daughter) / np.array(summed_tuple_Daughter).sum() if Normalization_Number_is_total_areas else np.array(summed_tuple_Daughter) / Normalization_Number

    # Calculating the errors and alphas of the newly combined data
    Alphas_of_Combined_Data = tuple(Daughter_Ratios / Parent_Ratios)
    Errors_of_Combined_Data = tuple((Daughter_Ratios/Parent_Ratios) * np.sqrt(
        (np.sqrt(np.array(summed_tuple_Daughter)) / np.array(summed_tuple_Daughter))**2 +
        (np.sqrt(np.array(summed_tuple_Parent)) / np.array(summed_tuple_Parent))**2))

    # Color themes for the Graph! Put """ """ around it if you want the regular default colors
    Line_Colors = ('#6dbb22', "#87CEEB") #
    Dot_Colors = ('#8c5c2f', "#8B4513") #
    Error_Bar_Colors = ('#4a7023', "#228B22")
    Color_Themes_Picker = np.array([0,1])
    Random_Theme = np.random.choice(Color_Themes_Picker)

    # Making the graph
    plt.errorbar(All_Isotopes, Alphas_of_Combined_Data, yerr = Errors_of_Combined_Data, capsize=3, marker='o', linestyle='-', color = Line_Colors[Random_Theme], ecolor = Error_Bar_Colors[Random_Theme], markerfacecolor = Dot_Colors[Random_Theme])
    plt.xlabel('Isotope')
    plt.ylabel('α')
    plt.title("Combined Data α Trends as a Function of Isotopic Composition")
    plt.grid(True)
    custom_labels = []
    for isotope in All_Isotopes:
      isotope_label = f"$\\mathrm{{^{{{isotope}}}{Atom_Names}}}$"
      custom_labels.append(isotope_label)  # Custom labels
    x_axis = All_Isotopes
    plt.xticks(x_axis, custom_labels)
    if not save_pdf:
      plt.show()

def Add_Combined_Data_Table_To_PDF(Normalization_Number = 1, Normalization_Number_is_total_areas = True, save_Table = False, PDF_MAKER = None, Pivot = False, Number_of_Decimals = None, additional_Information = None):
    """Generates a table with columns I defined for combined data for final graphs after using all_calculations_and_graphs multiple times or calculate_δα_indv multiple times.
     The table will have columns Total Areas Parent, Total Areas Daughter, α ε, δα, and preference. The rows will correspond to the Isotopes. Make an instance/put an instance of
     MakePDF in the PDF_Maker argument to put where it'll store the table. Put pivot as true if the data in the lists will be put in the table vertically as a column."""
    custom_labels = []
    for isotope in All_Isotopes:
      isotope_label = f"$\\mathrm{{^{{{isotope}}}{Atom_Names}}}$"
      custom_labels.append(isotope_label)
    # Sums values across all the tuples resulting in one tuple of all the summed areas.
    summed_tuple_Parent = tuple(sum(values) for values in zip(*All_Parent_Areas.values()))
    summed_tuple_Daughter = tuple(sum(values) for values in zip(*All_Daughter_Areas.values()))

    Combined_Error_Parent = np.sqrt(np.array(summed_tuple_Parent))
    Combined_Error_Daughter = np.sqrt(np.array(summed_tuple_Daughter))

    # Turns the tuples into an array to get the ratios which normalizes it
    Parent_Ratios = np.array(summed_tuple_Parent) / np.array(summed_tuple_Parent).sum() if Normalization_Number_is_total_areas else np.array(summed_tuple_Parent) / Normalization_Number
    Daughter_Ratios = np.array(summed_tuple_Daughter) / np.array(summed_tuple_Daughter).sum() if Normalization_Number_is_total_areas else np.array(summed_tuple_Daughter) / Normalization_Number

    # Calculating the errors and alphas of the newly combined data
    Alphas_of_Combined_Data = tuple(Daughter_Ratios / Parent_Ratios)
    Enrichments_of_Combined_Data = tuple((Daughter_Ratios/Parent_Ratios) - 1)
    Preferences = ['Parent' if ε < 0 else 'Neither' if ε == 0 else 'Daughter' for ε in Enrichments_of_Combined_Data]
    Errors_of_Combined_Data = tuple((Daughter_Ratios/Parent_Ratios) * np.sqrt(
        (np.sqrt(np.array(summed_tuple_Daughter)) / np.array(summed_tuple_Daughter))**2 +
        (np.sqrt(np.array(summed_tuple_Parent)) / np.array(summed_tuple_Parent))**2))

    #Round off Data to the decimal place specified in Number_of_Decimals if Number_of_Decimals is an integer
    if Number_of_Decimals is not None:
      summed_tuple_Parent = tuple(round(value, Number_of_Decimals) for value in summed_tuple_Parent)
      summed_tuple_Daughter = tuple(round(value, Number_of_Decimals) for value in summed_tuple_Daughter)
      Combined_Error_Parent = np.round(Combined_Error_Parent, Number_of_Decimals)
      Combined_Error_Daughter = np.round(Combined_Error_Daughter, Number_of_Decimals)
      Parent_Ratios = np.round(Parent_Ratios, Number_of_Decimals)
      Daughter_Ratios = np.round(Daughter_Ratios, Number_of_Decimals)
      Alphas_of_Combined_Data = tuple(round(value, Number_of_Decimals) for value in Alphas_of_Combined_Data)
      Enrichments_of_Combined_Data = tuple(round(value, Number_of_Decimals) for value in Enrichments_of_Combined_Data)
      Errors_of_Combined_Data = tuple(round(value, Number_of_Decimals) for value in Errors_of_Combined_Data)

    # Make the Table
    Table_of_Ultimate_Power = PDF_MAKER.define_table(custom_labels, np.array(['α', 'ε', 'Parent Error', 'Daughter Error', 'Error Bar', 'Preference']), np.array([Alphas_of_Combined_Data, Enrichments_of_Combined_Data, Combined_Error_Parent, Combined_Error_Daughter, Errors_of_Combined_Data, Preferences]), pivot = Pivot)
    if save_Table:
      Current_Table_Title = "Integrated Overview of All Data linked to the Same Isolation"
      PDF_MAKER.add_table(Table_of_Ultimate_Power, title  = Current_Table_Title, Additional_Information = additional_Information)
    if save_Table:
      return Table_of_Ultimate_Power

def overlaid_line_plot_EVI(save_pdf = False):
    """Makes an graph of all enrichment vs isotope data. This assumes you have used all_calculations_and_graphs multiple times or calculate_δα_indv multiple times and now want to plot all the data
    Put save_pdf = True if you are in the process of adding this chart to a pdf"""
    for key in All_enrichments:
        plt.errorbar(All_Isotopes, All_enrichments[key], yerr = All_Errors[key], label=key, capsize=3, marker='o', linestyle='-')
    plt.xlabel('Isotope')
    plt.ylabel('ε')
    plt.title("ε Trends vs. Isotopic Composition for Each Date")
    plt.legend(title = "Dates", fontsize='small', borderpad=0.3, labelspacing=0.2, loc='best')
    plt.grid(True)
    custom_labels = []
    for isotope in All_Isotopes:
      isotope_label = f"$\\mathrm{{^{{{isotope}}}{Atom_Names}}}$"
      custom_labels.append(isotope_label)  # Custom labels
    x_axis = All_Isotopes
    plt.xticks(x_axis, custom_labels)
    if not save_pdf:
       plt.show()

def overlaid_line_plot_AVI(save_pdf = False):
    """Makes an graph of all alpha vs isotope data. This assumes you have used all_calculations_and_graphs multiple times or calculate_δα_indv multiple times and now want to plot all the data
    Put save_pdf = True if you are in the process of adding this chart to a pdf"""
    for key in All_alphas:
        plt.errorbar(All_Isotopes, All_alphas[key], yerr = All_Errors[key], label=key, capsize=3, marker='o', linestyle='-')
    plt.xlabel('Isotope')
    plt.ylabel('α')
    plt.title("α Coefficient Trends as a Function of Isotopic Composition for Each Date")
    plt.legend(title = "Dates", fontsize='small', borderpad=0.3, labelspacing=0.2, loc='best')
    plt.grid(True)
    custom_labels = []
    for isotope in All_Isotopes:
      isotope_label = f"$\\mathrm{{^{{{isotope}}}{Atom_Names}}}$"
      custom_labels.append(isotope_label)  # Custom labels
    x_axis = All_Isotopes
    plt.xticks(x_axis, custom_labels)
    if not save_pdf:
      plt.show()

def all_calculations_and_graphs(molecule_name, name_of_atom_of_interest ,date_data_was_taken, molecular_weight, AMU, all_file_path, info_on_file_path,
                                row_starts_and_ends_parent, row_starts_and_ends_daughter, Bin_size, number_of_peaks, Normalization_Number,
                                parent_daughter, ROWSKIP = 7,  Drop_files = None, Print_Info = True, Print_Warnings = True, Print_Notices = True, Print_Errors = True, professional = False,
                                Normalization_Number_is_total_areas = False, rows_are_masses = False, graphs = False, save_PDF_Graph = False, save_PDF_Table = False, PDF_MAKER = None, Number_of_Decimals = None, Separate_files = False,
                                file_path_for_separate_file = None, separate_file_info_on_file_path = None, Debug = False):
    """This function will produce all the intended graphs and perform the calculations needed to generate graphs listed within this function for data taken on the same day. You'd use this multiple times
    to add data to the global variables then use the functions above to generate graphs for all data all together. These are graphs where you can only make graphs under the same isolation. To use the charts that
    use data from multiple different isolations, you'll need to use

    What this function does and explanation of the arguments:

    It first makes the class instance, then takes a string of the molecule name, a string of the name of the atom you're looking at, a string of the data it was taken, an integer of the molecular weight
    of the molecule, and an integer value of the AMU of the atom, and defines it for later use. Next, it takes the string of the file_path that has all the data files and takes the string of the link to a public google sheets
    or other method of getting the information pertaining to the data and defines the characteristics of that data. Third, it takes the rows and ends of parents and daughters in lists of where each peak starts and ends
    (can be only one list if the data is evenly spaced by bins), an integer of the bin size of how many rows from row starts (inclusive) it'll inlcude to row end (inclusive), an integer of the number_of peaks you are including in the
    calcultions, a integer Normalization_Number to what you'd want to normalize the data to (1 if none), parent_daughter which is the two NCE values you're comparing as a string in a list ['parent','daughter'], and ROWSKIP which
    you can specify to be equal to the amount of rows you want to skip to get to the headers of the data in the event that your data doesn't start at the first row (in this function you don't need to account for any
    indexing differences, just put the rows as they appear in the documents based on indexes or masses depending). Additionally, if you have bad data files you can always specify which files to drop
     like {'0': [1,2], '70': [8,9]} which works even if theres only one specified. You can also specify Print_Info = False if you do not want to print out every final calculation, Print_Warnings = False if you want to not
    print out any warnings (Things that do not cause errors, but may cause an error in later functions), Print_Errors = False if you don't want to print any errors (Things that do not work and break functionality due to inputs),
    Print_Notices = False if you do not want to print any notices (things that do not cause errors or require a warning, but may be unintended on the Users part), professional = True if you want the graphs to not have color themes and all be the same color,
    Normalization_Number_is_total_areas = True (if you are normalizing by total area then this will automatically normalize it by this for you), rows_are_masses = True (if you want to bin by starting masses and ending masses instead
    of indexing by row), and graphs = False (if you don't want to make any graphs).

    The PDF_MAKER argument is for an instance of MakePDF class of which you'd save any data to such as tables or graphs. If save_PDF_Graph or Save_PDF_Table is True, the graphs and/or tables will save respectively. For this function,
    it will output the tables and graphs I set it to within the function, for more customization you'll have to change it yourself. You can also specify to what decimal place the values will be rounded with Number_of_Decimals = integer of how many decimals you want in the tables.

    In the event you need to use calculate_δα_indv, but your Parent and Daughter files are in seperate folders with different NCE_Values, then you'll have to Put Seperate_files = True and include the new file path to the folder containing said value in file_path_for_seperate_file. Additionally, recall
    that if it's not the first and only instance of that NCE you'll need to also specify the isolation in ['Parent', 'Daughter'] like '20_391(28)' for example.
    You'll also need to provide a google_sheet/spreadsheet in the same format as before for the seperate_file in seperate_file_info_on_file_path.

    Set Debug = True to run any information during intermediate steps meant for debugging.

    The names can be edited in multiple ways such as making the data just a name if you want to manually take it apart for specific analyses, you can just copy paste
    the workflow and fill out the data and change variable names. Everything is also specified in more detail in the actual functions themselves.

    NOTE: There is a case where you'd need to do something easy that I did not implement. When you run this function it stores total areas in the global variable for future use with a key being the dates the data was taken.
    If you took data for parent or daughter on multiple days and the parent or daughter (whichever you didn't use first) is less than or greater than the number of days you took data,
    you'd still need to have a parent and daughter for this function to work. It would have repeats of the same data. In this event, you should manually clear the global variable for parent or daughter masses and total areas
    after using this function n amount of times, then run the function after you have an m number of files. By this I mean if you have 3 days where you took the data relating to the parent data and only took data for this daughter on one day, you'll
    run the function with separate files arguments twice, then set All_Daughter_Areas = {} and Daughter_Masses = {}. After this you run the last function still with the separate files arguments and now you'll
    have the data for the parent NCE from the three seperate days and the daughter NCE from the one day for data analysis. To reiterate, in this case n = 3 and m = 1, you clear the respective global variable when m = n dates.

    For further clarification, I am defining Error, Warning, and Notice as follows:
    Error: Anything that breaks the function it is in, where it would not be able to continue so it stops where the Error occurs
    Warning: Anything that does not break the function it is in, but it will likely break a diffrent function that relies on anything that is changed or outputted within that function
    Notice: Anything that does not break the function it is in and will likely not break any of the other functions, but may be unintended by the user that this occured or it won't function as intended due to it
    All of these assume that there is an issue with what the User puts in and not the functions themselves.

    Quick Guide to Arguments:
    molecule_name: A string of the name of the molecule. It will be formatted if entered a format like Nd(NO_{3})_{4}^{-}.
    name_of_atom_of_interest: A string of the name of the Atom you're observing, so for Nd(NO_{3})_{4}^{-} this would be 'Nd'
    date_data_was_taken: A string of the date you took the data in the format of 'MM/DD/YYYY'. If the Parent and Daughter were taken on seperate dates, you can enter both dates or only one.
    molecular_weight: An integer containing the molecular weight of the molecule before it was broken up in a mass spectrometer.
    AMU: An integer of mass in AMU for your atom_of_interest
    all_file_path: A string containing the path that takes you to the collected data. Formatting is important for this function, so ensure the columns have the specified names used in other functions.
    info_on_file_path: A string of the path that takes you to a googlesheet (change function if spreadsheet) containing the information formatted in the specified format in other functions.
    row_starts_and_ends_parent. A list of lists containg the index of the rows where you want to start collecting data and stop collection data such as [[start_1,end_1], [start_2,end_2]] for the 'Parent' molecule
    row_starts_and_ends_daughterA list of lists containg the index of the rows where you want to start collecting data and stop collection data such as [[start_1,end_1], [start_2,end_2]] for the 'Daughter' molecule
    Bin_size: A integer of how many rows will be included in one bin, you should account for this in your row_starts_and_ends.
    number_of_peaks: An integer of how many peaks are you looking at in the spectrum that you are including in the data (account for this in Bin_size and row_starts_and_ends.)
    Normalization_Number: An inyeger of what you want to normalize the peaks by, if you choose to normalize by total areas then this can be an arbitary integer and you set the optional argument to True
    parent_daughter: An list containing two strings containing what the parent and daughter NCE values are. For example, ['0','70']
    ROWSKIP: An integer of how many rows you'd need to skip in all_data_files to get to the header of the data. If none then put zero.

    Optional Arguments:
    Drop_files = None # Put in a dictionary with keys and and lists of what file numbers to drop
    Print_Info = True # To print calculations and general information
    Print_Warnings = True # Print anything that may cause Errors later
    Print_Notices = True # Print any information someone may want to know that isn't causing any errors, but may be an unintended use of the function on the users part
    Print_Errors = True
    professional = True
    Normalization_Number_is_total_areas = False
    rows_are_masses = False
    graphs = False # Show the graphs you generate. Otherwise it'll skip it
    save_PDF_Graph = False
    save_PDF_Table = False
    PDF_MAKER = None
    Number_of_Decimals = None
    Seperate_files = False.  If you want an NCE value that's in a different folder, set as True
    file_path_for_seperate_file = None
    seperate_file_info_on_file_path = None
    Debug = False

   """

    MSD = Mass_Spectrometer_Data() # Initialize a Mass_Spectrometer_Data class
    # Any attribute in MSD will be copied by The_Method class instance that is made in the Error_of_Data instance
    MSD.warnings = Print_Warnings # True by default
    MSD.notices = Print_Notices # True by default
    MSD.errors = Print_Errors # True by default
    MSD.define_molecule_characteristics(molecule_name, name_of_atom_of_interest ,date_data_was_taken, molecular_weight, AMU) # Define molecule characteristics
    if Separate_files:
      MSD_2 = Mass_Spectrometer_Data() # Make a second instance of this data class to get the files for the NCE in a Seperate_File
      MSD_2.warnings = Print_Warnings # True by default
      MSD_2.notices = Print_Notices # True by default
      MSD_2.errors = Print_Errors # True by default
      #Ensuring That MSD and MSD_2 Functions without Breaking due to missing values for the other NCE.
      MSD.define_all_file_path(all_file_path) # Gather the individual file paths from the path to the folder with all the files
      MSD_2.define_all_file_path(file_path_for_separate_file) # Gather the individual file paths from the path to the folder with all the files
      MSD.define_NCE(info_on_file_path, GoogleSheets = True) # Change this if it's not true for you
      MSD_2.define_NCE(separate_file_info_on_file_path, GoogleSheets = True) # Change this if it's not true for you
      MSD.only_include_CSV_files() # Only gathers CSV files
      MSD_2.only_include_CSV_files() # Only gathers CSV files
      MSD.define_NCE_Labels_and_Associated_Files() # Gathers the NCE labels from the google sheet/other spreadsheet and associates them to the file paths by indexes
      MSD_2.define_NCE_Labels_and_Associated_Files() # Gathers the NCE labels from the google sheet/other spreadsheet and associates them to the file paths by indexes
      MSD.get_file_paths()
      MSD_2.get_file_paths()
      try:
          # Case where Parent key is in MSD and Daughter key is in MSD_2. It identifies this by checking the specific NCE_ID made the define_NCE function in the MSD class.
          # Separate files will assume that the specific NCE wouldn't be in both as that is its purpose. There is no print but this would be self.warnings if it were in both since this works if NCE_ID is in MSD.
          MSD_KEY = parent_daughter[0] # Assumes MSD holds parent key
          MSD_2_KEY = parent_daughter[1] # Assumes MSD_2 holds daughter key
          Error = False # True if Parent key is not in MS
          # Compare Parent key to NCE_IDs and NCE. If it were '0' that means it's the first instance of '0' otherwise it'd be '0_391(28)', for example, which is 'NCE_ISOLATION'. However both files could have a zero, so isolation may matter
          if MSD_KEY in MSD.NCE: # Check if the key is in the NCE values
            if MSD_KEY in MSD_2.NCE: # Check if the NCE value also happens to be in MSD_2. If this is the case we have to check the isolation with the NCE value stored in the IDs
              # At this point in the code we can see that MSD 2 and MSD 1 have an NCE value like '0' with no specific isolation specified. So now we need to see if '0' for Parent key exists in MSD
              Index = MSD.NCE.index(MSD_KEY) # Get the appropriate index
              if MSD_KEY +"_"+ MSD.Isolations[Index] == MSD.nce_id[Index]: # Check if the isolation of the Key matches the NCE_id made using MSD instance data.
                pass # The Parent key is in MSD.NCE values and the isolation matches the NCE_ISOLATION IDs created for MSD. Therefore we can proceed with this try block
              else:
                Error = True # The Parent key is in MSD.NCE values, but after we compared the NCE_ID we find that they are not the same Isolation. So this means the parent key is in MSD_2
            else:
              pass # The Parent key is in MSD.NCE, but not MSD_2.NCE, so the Parent key is in MSD and we can continue with the try block
          else:
              Error = True # The Parent key is not in MSD.NCE values in any form
          if Error:
              raise ValueError("Parent key is not in MSD.NCE values")
          print('Parent NCE value is in the first folder, the daughter NCE value is in the separate folder') if Print_Notices or Debug else None
          if Drop_files is not None:
            if MSD_KEY in Drop_files:
              MSD.drop_files({MSD_KEY: Drop_files[MSD_KEY]})
            if MSD_2_KEY in Drop_files:
              MSD_2.drop_files({MSD_2_KEY: Drop_files[MSD_2_KEY]})
          if Debug:
            print(f"NCE for MSD before combining it: {MSD.NCE}")
            print(f"NCE for MSD_2 before combining it: {MSD_2.NCE}")
            print(f"numeric_file_indices for MSD beofre combining: {MSD.numeric_file_indices}")
            print(f"numeric_file_indices for MSD_2 beofre combining: {MSD_2.numeric_file_indices}")
            print(f"File names in MSD.file_names before combining: {MSD.file_names}")
            print(f"File names in MSD_2.file_names before combining: {MSD_2.file_names}")
            print(f"Dictionary with NCE keys and corresponding files in MSD before combining: {MSD.NCE_Labels_and_Files}")
            print(f"Dictionary with NCE keys and corresponding files in MSD_2 before combining: {MSD_2.NCE_Labels_and_Files}")
            print(f"The Dictionary containing the NCE keys and their corresponding file paths in MSD before combination: {MSD.NCE_File_Paths}")
            print(f"The Dictionary containing the NCE keys and their corresponding file paths in MSD_2 before combination: {MSD_2.NCE_File_Paths}")
          # Check and remove duplicates from MSD_2 before merging. We have confirmed the parent NCE is in MSD, but if there were a key of the same NCE and isolation in MSD 2, it would cause errors. To deal with this we do a check to remove the shared key from the daughter dictionary

          # Removes any similar keys from other since it would lead to duplications. However we also want to remove MSD_2 from MSD for that reason as if we didn't we'd get the wrong values when we combine them
          # Remove entire key from MSD_2.numeric_file_indices_dict if key exists in MSD.numeric_file_indices_dict, but keep the specified key MSD_2_KEY
          for key in list(MSD_2.numeric_file_indices_dict.keys()):
              if key in MSD.numeric_file_indices_dict and key != MSD_2_KEY:
                  print(f"Removing duplicate key '{key}' from MSD_2.numeric_file_indices_dict") if Debug else None
                  # Remove the key from MSD_2
                  del MSD_2.numeric_file_indices_dict[key]

                  # Also remove MSD_2_KEY from MSD if it exists
                  if MSD_2_KEY in MSD.numeric_file_indices_dict:
                      print(f"Also removing {MSD_2_KEY} from MSD.numeric_file_indices_dict") if Debug else None
                      del MSD.numeric_file_indices_dict[MSD_2_KEY]

          # Remove entire key from MSD_2.NCE_Labels_and_Files if key exists in MSD.NCE_Labels_and_Files, but keep the specified key MSD_2_KEY
          for key in list(MSD_2.NCE_Labels_and_Files.keys()):
              if key in MSD.NCE_Labels_and_Files and key != MSD_2_KEY:
                  print(f"Removing duplicate key '{key}' from MSD_2.NCE_Labels_and_Files") if Debug else None
                  # Remove the key from MSD_2
                  del MSD_2.NCE_Labels_and_Files[key]

                  # Also remove MSD_2_KEY from MSD if it exists
                  if MSD_2_KEY in MSD.NCE_Labels_and_Files:
                      print(f"Also removing {MSD_2_KEY} from MSD.NCE_Labels_and_Files") if Debug else None
                      del MSD.NCE_Labels_and_Files[MSD_2_KEY]

          # Remove entire key from MSD_2.NCE_File_Paths if key exists in MSD.NCE_File_Paths, but keep the specified key MSD_2_KEY
          for key in list(MSD_2.NCE_File_Paths.keys()):
              if key in MSD.NCE_File_Paths and key != MSD_2_KEY:
                  print(f"Removing duplicate key '{key}' from MSD_2.NCE_File_Paths") if Debug else None
                  # Remove the key from MSD_2
                  del MSD_2.NCE_File_Paths[key]

                  # Also remove MSD_2_KEY from MSD if it exists
                  if MSD_2_KEY in MSD.NCE_File_Paths:
                      print(f"Also removing {MSD_2_KEY} from MSD.NCE_File_Paths") if Debug else None
                      del MSD.NCE_File_Paths[MSD_2_KEY]

          # Merging MSD.NCE_Labels_and_Files
          merged_dict = {}
          for key in set(MSD.NCE_Labels_and_Files) | set(MSD_2.NCE_Labels_and_Files):  # Union of keys. Which is fine because they should be the exact same isolation
              if key in MSD.NCE_Labels_and_Files and key in MSD_2.NCE_Labels_and_Files:
                  # Merge and flatten the values
                  merged_dict[key] = MSD.NCE_Labels_and_Files[key] + MSD_2.NCE_Labels_and_Files[key]
              else:
                  # Take the value from whichever dictionary has the key
                  merged_dict[key] = MSD.NCE_Labels_and_Files.get(key, MSD_2.NCE_Labels_and_Files.get(key))
          MSD.NCE_Labels_and_Files = merged_dict
          print(f"MSD.NCE_Labels_and_Files: {MSD.NCE_Labels_and_Files}") if Debug else None

          # Merging MSD.NCE_File_Paths
          merged_dict = {} # Clear merged_dict from previous function
          for key in set(MSD.NCE_File_Paths) | set(MSD_2.NCE_File_Paths):  # Union of keys. As they have to be the same isolation so all files are valid to be put under the same key.
              if key in MSD.NCE_File_Paths and key in MSD_2.NCE_File_Paths:
                  # Merge and flatten the values
                  merged_dict[key] = MSD.NCE_File_Paths[key] + MSD_2.NCE_File_Paths[key]
              else:
                  # Take the value from whichever dictionary has the key
                  merged_dict[key] = MSD.NCE_File_Paths.get(key, MSD_2.NCE_File_Paths.get(key))
          MSD.NCE_File_Paths = merged_dict
          merged_numeric_indices = {}
          for key in set(MSD.numeric_file_indices_dict) | set(MSD_2.numeric_file_indices_dict):  # Union of keys
              if key in MSD.numeric_file_indices and key in MSD_2.numeric_file_indices:
                # Merge and flatten the values
                merged_numeric_indices[key] = MSD.numeric_file_indices_dict[key] + MSD_2.numeric_file_indices_dict[key]
              else:
                # Take the value from whichever dictionary has the key
                merged_numeric_indices[key] = MSD.numeric_file_indices_dict.get(key, MSD_2.numeric_file_indices_dict.get(key))
          MSD.numeric_file_indices_dict = merged_numeric_indices
          MSD.numeric_file_indices = MSD.numeric_file_indices_dict[MSD_KEY] + MSD.numeric_file_indices_dict[MSD_2_KEY]
          print(f"new numeric file indices: {MSD.numeric_file_indices}") if Debug else None
          print(f"Combined NCE File Path dictionary: {MSD.NCE_File_Paths}") if Debug else None
      except:
          print('Parent NCE value is in in the separate folder') if Print_Notices or Debug else None
          # Case where Daughter key is in MSD and Parent key is in MSD_2
          MSD_KEY = parent_daughter[1]
          MSD_2_KEY = parent_daughter[0]
          if Drop_files is not None:
            if MSD_KEY in Drop_files:
              MSD.drop_files({MSD_KEY: Drop_files[MSD_KEY]})
            if MSD_2_KEY in Drop_files:
              MSD_2.drop_files({MSD_2_KEY: Drop_files[MSD_2_KEY]})
          if Debug:
            print(f"NCE for MSD before combining it: {MSD.NCE}")
            print(f"NCE for MSD_2 before combining it: {MSD_2.NCE}")
            print(f"numeric_file_indices for MSD before combining: {MSD.numeric_file_indices}")
            print(f"numeric_file_indices for MSD_2 beofre combining: {MSD_2.numeric_file_indices}")
            print(f"File names in MSD.file_names before combining: {MSD.file_names}")
            print(f"File names in MSD_2.file_names before combining: {MSD_2.file_names}")
            print(f"Dictionary with NCE keys and corresponding files in MSD before combining: {MSD.NCE_Labels_and_Files}")
            print(f"Dictionary with NCE keys and corresponding files in MSD_2 before combining: {MSD_2.NCE_Labels_and_Files}")
            print(f"The Dictionary containing the NCE keys and their corresponding file paths in MSD before combination: {MSD.NCE_File_Paths}")
            print(f"The Dictionary containing the NCE keys and their corresponding file paths in MSD_2 before combination: {MSD_2.NCE_File_Paths}")
          # Check and remove duplicates from MSD_2 before merging. Although we confirmed parent is in MSD_2, there is still the possibility of the same NCE and isolation in MSD which would cause errors. To deal with this we do a check to that key from the daughter dictionary

          # Removes any similar keys from other since it would lead to duplications. However we also want to remove MSD from MSD_2 for that reason as if we didn't we'd get the wrong values when we combine them
          # Remove entire key from MSD.numeric_file_indices_dict if key exists in MSD_2.numeric_file_indices_dict, but keep the specified key MSD_KEY
          for key in list(MSD.numeric_file_indices_dict.keys()):
              if key in MSD_2.numeric_file_indices_dict and key != MSD_KEY:
                  print(f"Removing duplicate key '{key}' from MSD.numeric_file_indices_dict") if Debug else None
                  # Remove the key from MSD
                  del MSD.numeric_file_indices_dict[key]

                  # Also remove MSD_KEY from MSD_2 if it exists
                  if MSD_KEY in MSD_2.numeric_file_indices_dict:
                      print(f"Also removing {MSD_KEY} from MSD_2.numeric_file_indices_dict") if Debug else None
                      del MSD_2.numeric_file_indices_dict[MSD_KEY]

          # Remove entire key from MSD.NCE_Labels_and_Files if key exists in MSD_2.NCE_Labels_and_Files, but keep the specified key MSD_KEY
          for key in list(MSD.NCE_Labels_and_Files.keys()):
              if key in MSD_2.NCE_Labels_and_Files and key != MSD_KEY:
                  print(f"Removing duplicate key '{key}' from MSD.NCE_Labels_and_Files") if Debug else None
                  # Remove the key from MSD
                  del MSD.NCE_Labels_and_Files[key]

                  # Also remove MSD_KEY from MSD_2 if it exists
                  if MSD_KEY in MSD_2.NCE_Labels_and_Files:
                      print(f"Also removing {MSD_KEY} from MSD_2.NCE_Labels_and_Files") if Debug else None
                      del MSD_2.NCE_Labels_and_Files[MSD_KEY]

          # Remove entire key from MSD.NCE_File_Paths if key exists in MSD_2.NCE_File_Paths, but keep the specified key MSD_KEY
          for key in list(MSD.NCE_File_Paths.keys()):
              if key in MSD_2.NCE_File_Paths and key != MSD_KEY:
                  print(f"Removing duplicate key '{key}' from MSD.NCE_File_Paths") if Debug else None
                  # Remove the key from MSD
                  del MSD.NCE_File_Paths[key]

                  # Also remove MSD_KEY from MSD_2 if it exists
                  if MSD_KEY in MSD_2.NCE_File_Paths:
                      print(f"Also removing {MSD_KEY} from MSD_2.NCE_File_Paths") if Debug else None
                      del MSD_2.NCE_File_Paths[MSD_KEY]

          # Merging MSD_2.NCE_File_Paths
          merged_dict = {}
          for key in set(MSD_2.NCE_File_Paths) | set(MSD.NCE_File_Paths):  # Union of keys
              if key in MSD_2.NCE_File_Paths and key in MSD.NCE_File_Paths:
                  # Merge and flatten the values
                  merged_dict[key] = MSD_2.NCE_File_Paths[key] + MSD.NCE_File_Paths[key]
              else:
                  # Take the value from whichever dictionary has the key
                  merged_dict[key] = MSD_2.NCE_File_Paths.get(key, MSD.NCE_File_Paths.get(key))
          MSD.NCE_File_Paths = merged_dict

          # Merging Numeric_File_Indices
          merged_numeric_indices = {}
          for key in set(MSD_2.numeric_file_indices_dict) | set(MSD.numeric_file_indices_dict):  # Union of keys
              if key in MSD_2.numeric_file_indices and key in MSD.numeric_file_indices:
                # Merge and flatten the values
                merged_numeric_indices[key] = MSD_2.numeric_file_indices_dict[key] + MSD.numeric_file_indices_dict[key]
              else:
                # Take the value from whichever dictionary has the key
                merged_numeric_indices[key] = MSD_2.numeric_file_indices_dict.get(key, MSD.numeric_file_indices_dict.get(key))
          MSD.numeric_file_indices_dict = merged_numeric_indices
          MSD.numeric_file_indices = MSD.numeric_file_indices_dict[MSD_2_KEY] + MSD.numeric_file_indices_dict[MSD_KEY]
          print(f"new numeric file indices: {MSD.numeric_file_indices}") if Debug else None
          print(f"Combined NCe file path dictionary: {MSD.NCE_File_Paths}") if Debug else None
    else:
      # All Files for specified NCEs are in the same folder
      print('Parent and Daughter NCE values are in the same folder') if Print_Notices or Debug else None
      MSD.define_all_file_path(all_file_path) # Gather the individual file paths from the path to the folder with all the files
      MSD.define_NCE(info_on_file_path, GoogleSheets = True) # Change this if it's not true for you
      MSD.only_include_CSV_files() # Only gathers CSV files
      MSD.define_NCE_Labels_and_Associated_Files() # Gathers the NCE labels from the google sheet/other spreadsheet and associates them to the file paths by indexes
      if Drop_files is not None:
          MSD.drop_files(Drop_files) # Drops files for any of the NCEs if specifed
      MSD.get_file_paths() # Gathers all the file paths for the defined NCEs and makes a dictionary where each NCE has defined file paths
    MSD.merge__and_store_files_by_nce(MSD.NCE) # Makes a big data frame by combining all data with same values together, but its use here is to ensure functionality as it has multiple uses and needs to be defined for some of the functions used here to work
    Error_of_Data = Error_Propagation(MSD) # Initalize a Error class taking in the data from the MSD instance
    Error_of_Data.print_information = Print_Info # True by default, will print out all calculations
    if rows_are_masses: # If you chose to index by mass instead of the row indexes directly, otherwise it'll continue with indexing by rows
      Error_of_Data.calculate_δα_indv(row_starts_and_ends_parent, row_starts_and_ends_daughter, Bin_size, number_of_peaks, Normalization_Number, parent_daughter, ROWSKIP = 7, NBTA = Normalization_Number_is_total_areas, rows_are_masses = True)
    else:
      Error_of_Data.calculate_δα_indv(row_starts_and_ends_parent, row_starts_and_ends_daughter, Bin_size, number_of_peaks, Normalization_Number, parent_daughter, ROWSKIP = 7, NBTA = Normalization_Number_is_total_areas)
    if save_PDF_Table:
      custom_labels = []
      for isotope in All_Isotopes:
        isotope_label = f"$\\mathrm{{^{{{isotope}}}{Atom_Names}}}$"
        custom_labels.append(isotope_label)  # Custom labels for Isotopes
      if Separate_files:
        # Make title distinction because the default title assumes all the data was in the same folder taken on the same day
        Current_Table_Title = f"Tabulated Computational Outcomes for {MSD.molecule_name} from Data taken on Different Dates"
      else:
        Current_Table_Title = f"Tabulated Computational Outcomes for {MSD.molecule_name} Based on {MSD.date_data_was_taken} Data"
      PDF_MAKER.add_table(PDF_MAKER.define_table(custom_labels, np.array(['Alpha','Enrichment','Error Bar', 'Parent_Ratio', 'Daughter_Ratio']), [Error_of_Data.alphas, Error_of_Data.enrichments, Error_of_Data.δα, Error_of_Data.Parent_ratios, Error_of_Data.Daughter_ratios], pivot = True, number_of_decimals = Number_of_Decimals), title = Current_Table_Title,
                          Additional_Information = f"The NCEs are Parent: {parent_daughter[0]} {'|'} Daughter: {parent_daughter[1]}") #Table totals
    if graphs:
      # To make the graphs if graphs is True
      Graphing_Molecule_Data = Graphs(MSD, Error_of_Data) # Intalize a Graphing class that takes in the MSD and Error_of_Data instances
      """Graphing_Molecule_Data.professional = professional # False by default, will make all graphs the same color
      Graphing_Molecule_Data.αVIs(save_pdf = save_PDF_Graph) # Create alpha versus isotope graph
      Graphing_Molecule_Data.εVIs(save_pdf = save_PDF_Graph) # Create enrichment versus isotope graph
      Graphing_Molecule_Data.RIVM()""" # Create Relative intensity versus masses graph
      Graphing_Molecule_Data.generate_all_IVM(387.5,400,340,355,['0','70']) #You can change the bounds of the data with xlim left and right as well as change which NCE they are for within this function
    elif save_PDF_Graph:
      # Save the Graphs to the PDF if save_PDF_Graph is True. Same Descriptions as in Graphs
      Graphing_Molecule_Data = Graphs(MSD, Error_of_Data) # Intalize a Graphing class that takes in the MSD and Error_of_Data instances
      Graphing_Molecule_Data.professional = professional # False by default, will make all graphs the same color
      PDF_MAKER.add_chart(Graphing_Molecule_Data.αVIs, save_pdf = save_PDF_Graph)
      PDF_MAKER.add_chart(Graphing_Molecule_Data.εVIs, save_pdf = save_PDF_Graph)
      Graphing_Molecule_Data.RIVM(PDF_Maker = PDF_MAKER)
      #PDF_MAKER.add_chart(Graphing_Molecule_Data.generate_all_IVM, 387.5,400,340,355,['0','70'])

def export_dataframe(data_frame, name):
    """Takes in any data frame or data you want to export and saves it in the file format you specify"""
    filename = name + '.csv'
    data_frame.to_csv(filename)
    files.download(filename)

#Classes
class Mass_Spectrometer_Data:
  """Allows you to connect all the classes and data_files"""
  def __init__(self):
    self.molecule_name = "[Dy(NO_{3})_{4}]^{-}" #Example, the format doesn't matter, I just prefer it this way.
    self.atom_name = 'Dy'
    self.date_data_was_taken = "11/14/1987"
    self.molecular_weight_of_molecule = 410.516 # g/mol. This is just an example
    self.AMU_Element = 162.5
    self.all_files_path = "path" # A string of the Path to folder containing every CSV file
    self.info_on_files_path_or_URL = "pathh" # A string of the path to a CSV file of all info about the files. This is specific to the project. If google sheets,this should be a URL.
    self.info_on_files_data = None
    self.file_names = []
    self.NCE = []
    self.Isolations = []
    self.NCE_Labels_and_Files = {} # A dictionary that shows which files belong to which NCE values. This does not specify isolation.
    self.numeric_file_indices = []
    self.NCE_File_Paths = {} # A dictionary containing the key of the NCE values and the value pair of a list of file path names corresponding to each file.
    self.print_information = False # Set to True if you want to print out relevant information you have. This does not include error messages.
    self.errors = True #Set to False if you don't want to print out any Errors.
    self.warnings = True # Set to False if you don't want to print out any warnings.
    self.notices = True # Set to False if you don't want to print out any notices
    self.NCE_dataframes = {} # For the event that you want to just store all data into a data frame
    self.nce_id = 'ISOUPTOPE' # A list of the unique NCE IDs
    self.The_Lorax= [] # Has all the unique NCE IDs in the order they were found
    self.unsorted_NCE = [] # A list of the NCE value for every file.
    self.unsorted_Isolations = [] # A list of the Isolation for every file.
    self.unsorted_NCE_ID = [] # A list of the NCE ID for every file.
    self.numeric_file_indices_dict = {} # Puts the number of the files

  def define_molecule_characteristics(self, molecule_name, name_of_atom_of_interest ,date_data_was_taken, molecular_weight, AMU):
    """Takes the of the molecule that you analyzed in the Mass spectrometer as a string, the name of the specific atom of the molecule that is of interest/all ligands are
    bound to as a string, the date of the analysis in the format of MM/DD/YEAR as a string, the molecular weight of the molecule as a integer, and the AMU of the element
    (such as Dy or Nd) from the periodic table as a integer."""
    try:
        self.molecule_name = format_chemical_formula(molecule_name)
    except:
      self.molecule_name = molecule_name
    self.atom_name = name_of_atom_of_interest
    self.date_data_was_taken = date_data_was_taken
    self.molecular_weight_of_molecule = molecular_weight
    self.AMU_Element = AMU

  def define_all_file_path(self, file_path):
    """Takes in a string of the file_path that contains every single file you want to use for data analysis. This path should contain every single CSV file you want to use"""
    self.all_files_path = file_path


  def define_NCE(self, info_on_files_path_or_URL, **Manually):
    """For this you can either put in a string of the file_path of a file with every NCE labled in a column with the associated file numbers
     and let the code sort it out or add an additional arguments Manual = True and NCE_values = [put values in here]. The intended file_path has columns labeled
     file number, isolation, NCE, and comment. file number is the number of the associated file such as file 0, file 1, etc. Isolation is the associated isolation
     for that file in the format of isolated mass(bin width). NCE is the associated normalized collision energy used in the mass spectrometer. The comment is not
     needed for the code, but you should examine it to deicde whether to that file later on in events such as running out of spray or forgetting to change the range you used;
     the comment column is not needed for the code, but you could have one to decide whether the data isn't good to use for statisitcal analysis or not."""

    CSV, Excel, GoogleSheets, Manual = (Manually.get(key, False) for key in ['CSV', 'Excel', 'GoogleSheets', 'Manual']) # Extracts optional parameters or set default False values
    self.info_on_files_path_or_URL = info_on_files_path_or_URL

    # Load the appropriate file type
    if CSV:
        self.info_on_files_data = pd.read_csv(info_on_files_path_or_URL)
    elif Excel:
        self.info_on_files_data = pd.read_excel(info_on_files_path_or_URL)
    elif GoogleSheets:
        self.info_on_files_data = import_google_spreadsheet_file(info_on_files_path_or_URL)
    else:
        print('You need to specify what type of file it is (CSV, Excel, or GoogleSheets)') if self.errors else None
        return  # Exit the function if file type is not specified
    NCE_values = []
    if Manual:  # If Manual is True, retrieve the provided NCE values
        NCE_values = Manually.get('NCE_values', [])
    else:
        # Otherwise, extract NCE values from the data
        data = self.info_on_files_data
        nce_columns = {'NCE', 'nce', 'nce ', ' nce', 'NCE ', ' NCE'}
        file_columns = {'file', 'file ', ' file', 'file #', 'file#', 'file number', 'File', 'file_number', 'File_Number', 'File_number', 'File_#', 'File#', 'File #', 'File ', ' File'}
        isolations_and_width = {'Isolation', 'isolation', ' isolation', ' isolation', ' Isolation', 'Isolation '}
        data.columns = ['NCE' if col in nce_columns else 'file number' if col in file_columns else 'isolation' if col in isolations_and_width else col
        for col in data.columns]

        if not any(col in data.columns for col in nce_columns) or not any(col in data.columns for col in file_columns) or not any(col in data.columns for col in isolations_and_width):
            print("Error: 'NCE' or 'file number' or 'isolation' column missing from data.") if self.error else None
            return {'>:('}

        self.info_on_files_data = data # To save the changed names for consistency
        if 'NCE' in data.columns:
            x = data['NCE'].tolist()
            y = data['isolation'].tolist()
            self.unsorted_NCE = x # To store NCE for each file so it can be used for indexing in different functions
            self.unsorted_Isolations = y # To store NCE for each file so it can be used for indexing later in different functions
            nce_id = [lambda t = t: str(x[t]) + "_" + str(y[t]) for t in range(len(x))]
            nce_ids = [func() for func in nce_id] #NCE_IDs for every single file
            self.unsorted_NCE_ID = nce_ids # To store NCE for each file so it can be used for indexing later in different functions
            seen = set()
            nce_ids = [x for x in nce_ids if not (x in seen or seen.add(x))] #Unique NCE_IDs
            print(f"All unique NCE and Isolations: {nce_ids}") if self.print_information else None
            # Ensures unique values while preserving order
            The_Lorax = [] # Protects the unique NCE values like he protected the trees
            Isolations = [] #Holds Isolations for any unique value
            # If a NCE value is the first instance of itself, it'll be just that number. If it repeats and the isolations are not the same, it'll add a new NCE value as NCE_ISOLATION
            for i in range(len(x)):
                nce_isolation_id = str(x[i]) + "_" + str(y[i])  # Unique identifier as 'NCE_isolation'
                if str(x[i]) not in NCE_values:
                    NCE_values.append(str(x[i]))  # Append as a string
                    The_Lorax.append(nce_isolation_id)
                    Isolations.append(str(y[i]))
                elif (str(x[i]) + ' ' + str(y[i])) not in NCE_values and nce_isolation_id not in The_Lorax:
                      NCE_values.append(nce_isolation_id)
                      The_Lorax.append(nce_isolation_id)
                      Isolations.append(str(y[i]))
            # Removes duplicate entries from the final list
            NCE_values = list(dict.fromkeys(NCE_values))  # Keeps the order and removes duplicates

    # Optionally print the NCE values if requested
    print(f"NCE Values: {NCE_values}") if self.print_information else None
    self.nce_id = nce_ids # NCE IDs
    self.NCE = NCE_values # NCE Values where if it is the first instance of the NCE value it'll be that value, but if it's not it'll be NCE_ISOLATION. This also has no duplicates and is ordered in the order they're found
    self.The_Lorax = The_Lorax # NCE IDs with preserved order and holds unique values only
    self.Isolations = Isolations # Isolations for each Unique NCE value. This is ordered

  def only_include_CSV_files(self):
    """This function may not be necessary for all, however if your folder contains other files that aren't CSV files of the data you want to use for the data analysis, this function
    will sort through the files from self.all_files_path and know to only use the CSV files"""
    try:
            # List all files in the specified directory
            all_files = os.listdir(self.all_files_path)

            # Filter to include only CSV files
            self.file_names = [file for file in all_files if file.endswith('.csv')]

            # Optional: Print the list of included CSV files
            print("Included CSV files:", self.file_names) if self.print_information else None

    except FileNotFoundError:
            print(f"The directory '{self.all_files_path}' does not exist.") if self.errors else None
    except Exception as womp:
            print(f"An error occurred: {womp}") if self.warnings else None

  def define_NCE_Labels_and_Associated_Files(self):
    """Associates file names with NCE values (0 or 70 or 0(isolation)) based on the 'NCE' column in the info_on_files_data.
    Creates a numeric list from file names for proper indexing, ensuring that missing files are handled appropriately.
    ensure the file is named in the following format: filename_####.csv, as to properly map file names to file numbers,
    it'll drop the .csv and look at the digits after the _ ."""

    # Create a numeric list from file names by extracting numbers before the first underscore
    numeric_file_indices = []
    for filename in self.file_names:
        # Split the filename to extract the number before the underscore and remove '.csv'
        num_part = filename.split('_')[1].replace('.csv', '') if '_' in filename else ''
        # Convert to integer
        numeric_file_indices.append(int(num_part))
    self.numeric_file_indices = numeric_file_indices

    # Initialize dictionaries for storing files based on plain NCE and formatted NCE
    initial_nce_files = {nce: [] for nce in self.NCE}
    Same_NCE_different_ISOLATION = []
    for ID in self.nce_id:
      if ID in self.NCE:
        Same_NCE_different_ISOLATION.append(ID)
    data = self.info_on_files_data
    x = data['NCE'].tolist()
    y = data['isolation'].tolist()

   # Populate NCE_ID with files, using formatted `nce_key` as keys
    for idx, row in self.info_on_files_data.iterrows():
        nce_value = str(row['NCE'])
        isolation = str(row['isolation'])
        nce_key = f"{nce_value}_{isolation}"
        file_number = int(row['file number'])

        # Match file_number with numeric_file_indices and retrieve the filename
        if file_number in numeric_file_indices:
            index_in_list = numeric_file_indices.index(file_number)
            filename = self.file_names[index_in_list]
            if nce_key in Same_NCE_different_ISOLATION:
                initial_nce_files[nce_key].append(filename)
            else:
                initial_nce_files[nce_value].append(filename)
        else:
            print(f"Notice: File number {file_number} not found in defined file names.") if self.notices else None

    self.NCE_Labels_and_Files = initial_nce_files

    # Create a dictionary for numeric file indices associated with each NCE key
    numeric_indices_per_nce = {}
    for nce_key, filenames in self.NCE_Labels_and_Files.items():
        # Extract numeric indices for each file in the current NCE
        numeric_indices = []
        for filename in filenames:
            # Extract the numeric part of the filename (after '_')
            num_part = filename.split('_')[1].replace('.csv', '') if '_' in filename else ''
            numeric_indices.append(int(num_part))
        numeric_indices_per_nce[nce_key] = numeric_indices

    self.numeric_file_indices_dict = numeric_indices_per_nce

    # Optionally print the final result for verification
    print(f"NCE Labels and Files: {self.NCE_Labels_and_Files}") if self.print_information else None

    return self.NCE_Labels_and_Files

  def drop_files(self, NCE_values_and_which_files_to_drop):
    """
    Removes specified files from the dataset based on NCE values and file numbers provided in a dictionary format.
    Args:
        NCE_values_and_which_files_to_drop (dict): Dictionary where keys are NCE values (e.g., '0', '70')
        and values are lists of file numbers to drop (e.g., {'0': [3, 6], '70': [5, 7]}).
    """
    # Create a mapping from filename to the extracted file index
    file_index_map = {file_name: int(file_name.split('_')[1].split('.')[0]) for file_name in self.file_names}

    for NCE_value, file_numbers_to_drop in NCE_values_and_which_files_to_drop.items():
        if NCE_value in self.NCE_Labels_and_Files:
            # Convert file numbers to corresponding filenames based on the mapping
            files_to_drop = [file_name for file_name, index in file_index_map.items() if index in file_numbers_to_drop]

            # Filter out the files that are not in the drop list
            updated_files = [file for file in self.NCE_Labels_and_Files[NCE_value] if file not in files_to_drop]

            self.numeric_file_indices = [index for index in self.numeric_file_indices if index not in file_numbers_to_drop]

            # Update the dictionary directly
            self.NCE_Labels_and_Files[NCE_value] = updated_files

            if self.print_information:
                print(f"Updated NCE {NCE_value}: {updated_files}")
        else:
            print(f"NCE {NCE_value} not found in the dictionary.") if self.warnings else None

    """Test Cases and Workflow for clarity:

    # First the class would be initialized
    obj = Mass_Spectrometer_Data()

    Then lets say you ended up with self.NCE_Labels_and_Files = {
            '0': [1, 2, 3, 4, 5, 6],
            '70': [10, 20, 30, 40],
            '90': [100, 200, 300]}

    # Before dropping files
    print("Before:", obj.NCE_Labels_and_Files)

    # Drop files 3 and 6 from NCE 0
    obj.drop_files(**{'0': [3, 6]})

    # After dropping files
    print("After:", obj.NCE_Labels_and_Files)
    Expected Output:

    This would be the expected output:

    Before: {'0': [1, 2, 3, 4, 5, 6], '70': [10, 20, 30, 40], 90: [100, 200, 300]}
    Updated NCE '0': [1, 2, 4, 5]
    After: {'0': [1, 2, 4, 5], '70': [10, 20, 30, 40], '90': [100, 200, 300]}

    Note: the outputted numbers would be the file names, so 1 would be something like data_001.csv

    """

  def get_file_paths(self, reverse_order=False):
    """Associates each file number with a string of a file path from info_on_files_data
    and returns a dictionary with file paths instead of file numbers. This function should be
    used after NCE_Labels_and_Files is defined and drop_files is used if necessary.

    Parameters:
    reverse_order (bool): If True, the dictionary value is reversed to correctly map the file_paths.
    It will correctly reverse the order back after mapping the corresponding file_paths.
    Use this if the folder has the files from the last file at the top to the first file at the bottom.
    """
    NCE_File_Paths = {}

    # Gets the list of files from the directory
    files = os.listdir(self.all_files_path)
    # Sorts the files to ensure they are in the correct order
    files.sort()  # Ensure files are sorted correctly

    #print("Files in directory:", files)  # Debug print to show the files

    for NCE_value, file_names in self.NCE_Labels_and_Files.items():
        file_paths = []
        #print(f"Processing NCE: {NCE_value} with files: {file_names}")  # Debug print for current NCE and files

        for file_name in file_names:  # Iterate through the actual file names
            if file_name in files:  # Check if the file exists in the directory
                file_path = os.path.join(self.all_files_path, file_name)
                file_paths.append(file_path)
            else:
                print(f"Notice: File '{file_name}' not found in the directory.") if self.notices else None

        if reverse_order:
            # Reverses the order of the file paths if needed
            file_paths.reverse()  # This restores the original order if needed

        NCE_File_Paths[NCE_value] = file_paths  # Stores the list of file paths for each NCE value

        print(f"NCE File Paths for {NCE_value}: {file_paths}") if self.print_information else None

    #print(f"Final NCE File Paths: {NCE_File_Paths}")  # Debug print to show the final paths
    self.NCE_File_Paths = NCE_File_Paths
    return NCE_File_Paths

  def merge__and_store_files_by_nce(self, nce_values=None, skip_rows = 7):
        """Merges all the file_paths from Mass_Spectrometer_Data.NCE_File_Paths and returns a dictionary containing the Pandas data frames.
        Takes in NCE values in a list of the values you want to merge:

        # Option 1: Merges files for all NCEs (default behavior)
        merged_dataframes_all = the_method_over_all_files.merge_files_by_nce()

        # Option 2: Merges files only for specified NCE values (e.g., NCE '0' and '70')
        merged_dataframes_selected = the_method.merge_files_by_nce(nce_values=['0', '70'])

        And you can access it by doing this:
        Access merged DataFrame for NCE '0'
        nce_0_df = NCE_dataframes_selected.get('0')"""

        merged_dataframes = {}

        # Ensure that self.Data has the NCE_File_Paths attribute
        if not hasattr(self, 'NCE_File_Paths'):
            raise AttributeError("self does not have an attribute 'NCE_File_Paths'.") if self.errors else None

        # If no NCE values are specified, use all keys from the NCE_File_Paths dictionary
        if nce_values is None:
            nce_values = list(self.NCE_File_Paths.keys())

        # Loop through the specified NCE values and merge their corresponding files
        for nce in nce_values:
            if nce in self.NCE_File_Paths:  # Ensure NCE exists in the dictionary
                df_list = []

                for file_path in self.NCE_File_Paths[nce]:
                  try:
                      # Handling of bad lines
                      df = pd.read_csv(file_path, on_bad_lines='skip', skiprows = skip_rows)  # Skip problematic lines.
                      df_list.append(df)  # Add DataFrame to list
                  except pd.errors.ParserError as e:
                      print(f"ParserError: Problem reading {file_path}: {e}") if self.warnings else None
                  except Exception as e:
                      print(f"Error reading {file_path}: {e}") if self.warnings else None
                # Concatenates all DataFrames in the list for the current NCE
                if df_list:
                    merged_dataframes[nce] = pd.concat(df_list, ignore_index=True)
                else:
                    print(f"No valid data for NCE {nce}") if self.warnings else None
            else:
                print(f"NCE value '{nce}' not found in the NCE_File_Paths dictionary.") if self.warnings else None

        self.NCE_dataframes = merged_dataframes
        return merged_dataframes

class The_Method(Mass_Spectrometer_Data):
  """Draws comparisions among two different files of different NCE only"""
  def __init__(self):
    self.raw_data_1 = None
    self.raw_data_2 = None
    self.isotopes_1 = []
    self.isotopes_2 = [] #You get Isotopes from the parent, but I included this in case it's desired.
    self.masses_P = []
    self.masses_D = []
    self.areas_1 = []
    self.areas_2 = []
    self.ratios_1 = []
    self.ratios_2 = []
    self.alphas = []
    self.enrichments = []
    self.YN_list = []
    self.preference_list = []
    self.print_information = False

  def copy_from(self, Mass_Spectrometer_Data_instance):
        # Copy attributes from another Mass_Spectrometer_Data instance, assuming you want to use The_Method with the files from Mass_Spectrometer_Data.
        self.molecule_name = Mass_Spectrometer_Data_instance.molecule_name
        self.atom_name = Mass_Spectrometer_Data_instance.atom_name
        self.date_data_was_taken = Mass_Spectrometer_Data_instance.date_data_was_taken
        self.molecular_weight_of_molecule = Mass_Spectrometer_Data_instance.molecular_weight_of_molecule
        self.AMU_Element = Mass_Spectrometer_Data_instance.AMU_Element
        self.all_files_path = Mass_Spectrometer_Data_instance.all_files_path
        self.info_on_files_path_or_URL = Mass_Spectrometer_Data_instance.info_on_files_path_or_URL
        self.info_on_files_data = Mass_Spectrometer_Data_instance.info_on_files_data
        self.file_names = Mass_Spectrometer_Data_instance.file_names
        self.NCE = Mass_Spectrometer_Data_instance.NCE
        self.NCE_Labels_and_File_Numbers = Mass_Spectrometer_Data_instance.NCE_Labels_and_Files
        self.numeric_file_indices = Mass_Spectrometer_Data_instance.numeric_file_indices
        self.NCE_File_Paths = Mass_Spectrometer_Data_instance.NCE_File_Paths
        self.print_information = Mass_Spectrometer_Data_instance.print_information
        self.errors = Mass_Spectrometer_Data_instance.errors
        self.warnings = Mass_Spectrometer_Data_instance.warnings
        self.notices = Mass_Spectrometer_Data_instance.notices
        self.NCE_dataframes = Mass_Spectrometer_Data_instance.NCE_dataframes

  def load_data(self, csv_file_1, csv_file_2, row_skip):
   """"Load the data from the two CSV files you are going to compare into the Pandas Dataframe. Takes in two strings of the file_paths for the files and an integer for how many rows to skip. This is for the event that you don't
   want to use the parent class Mass_Spectrometer_Data."""
   self.raw_data_1 = pd.read_csv(csv_file_1, skiprows = row_skip) # If the file has text other than the data, put the number of rows needed to skip to get to the data. Ensure all files are uniform
   self.raw_data_2 = pd.read_csv(csv_file_2, skiprows = row_skip)

  def calculate_areas_and_isotopes(self, row_start, row_end, bin_size, file_number, ROWSSKIPPED = 7, Calculate_Isotopes_and_Masses = False, rows_are_masses = False):
    """ This will calculate areas and isotopes from the mass spectrometer data given the specifications. bin = int, row_start = int, row_end = int where the bin is a group of values you want to group together for each peak,
    row_start and row_end are respectively where you'll start grouping data and where the last row will be. It is inclusive of the value you put in. Finally, you need to put which file you're calculating
    the areas for, file = 1 or 2 for the respective files since the parameters of each is different. One note is that your rows should correspond to the row number in either excel or google sheets after deleting any rows before the headers and data. The indexing and reported bins values are specifically tailored for this.
    Calculate Isotopes = False if you want the function to calculate areas only, set it to True if you need the isotope values. If you index by mass, ROWSSKIPPED is needed to specify how
    many rows you'd skip in the original spreadsheet to get to the data and rows_are_mass must = True"""

    # Select the correct DataFrame based on the file number
    data = self.raw_data_1 if file_number == 1 else self.raw_data_2

    # You can change these if you use something that didn't originate from google sheets or excel
    pandas_difference_index = 1 # Pandas uses 0-Based indexing whereas google sheets uses 1-Based indexing, so the answer would be one index off. This accounts for that
    header_affect_on_index = 1 # The index is also offset by the header, so this accounts for it being off by 1

    if rows_are_masses:
       masses_to_find = [row_start, row_end]

       # Define tolerance for approximate matching due to floating decimals between files. Change if necessary for more precise matching
       if "." in str(masses_to_find[0]):
          decimal_part = str(masses_to_find[0]).split(".")[1]
          if len(decimal_part) >= 5:
            tolerance = 1e-5  # Match up to the fifth decimal place
          else:
            n = len(decimal_part)
            tolerance = 10 ** -n  # To the nth decimal place
       else:
          tolerance = 1e-5  # Default tolerance if no decimal part exists

       for m in masses_to_find:

          print(f"Mass {m}: Matches: {data['Mass'].apply(lambda x: abs(x - m) < tolerance).sum()}")

       # Find row indices where the "Mass" values are within the tolerance
       row_indice = data[data['Mass'].apply(lambda x: any(abs(x - m) < tolerance for m in masses_to_find))].index.tolist()
       row_indices = []
       for index in row_indice:
         row_indices.append(index + ROWSSKIPPED + header_affect_on_index + pandas_difference_index)
       print(f"rows_indicies: {row_indices}") #Debugger
       row_start = row_indices[0]
       row_end = row_indices[1]
       #Debugger in the event you want to see what the actual resulting indexes are. This would be needed if there are many decimals.
       print(f"row_start: {row_start}")
       print(f"row_end: {row_end}")

    # Ensure row_start and row_end are within valid bounds
    if row_start < 0 or row_end >= len(data) or row_start > row_end:
        print(f"Invalid row range: row_start={row_start}, row_end={row_end}, max_row={len(data)}") if self.warnings else None
        return []

    areas = []  # List to hold area sums
    isotopes, masses = [], [] if Calculate_Isotopes_and_Masses else None # Lists to hold isotope and masses values

    # Loops through the DataFrame until row_start exceeds row_end
    current_bin_start = row_start
    while current_bin_start < row_end:
        # To define the end of the current bin
        current_bin_end = current_bin_start + bin_size

        # To ensure the current bin does not exceed row_end
        if current_bin_end > row_end :
            print(f"Warning: Adjusting bin size as it exceeds the specified row_end at current_bin_start={current_bin_start}.") if self.warnings else None
            current_bin_end = row_end + (current_bin_end - row_end) # Adjust to include the values that fit within the specified bin

        # Select the current bin of rows
        bin_data = data.iloc[current_bin_start- pandas_difference_index - header_affect_on_index:current_bin_end - pandas_difference_index -  header_affect_on_index  + 1]  # +1 to include current_bin_end

        # Debugging statement
        #print(f"Current Bin: Start Row = {current_bin_start}, End Row = {current_bin_end}")

        # Ensure bin_data is not empty
        if bin_data.empty:
            print("Warning: bin_data is empty at current_bin_start:", current_bin_start) if self.warnings else None
            break

        # Sum the Intensities for Area
        bin_sum = bin_data['Intensity'].sum()
        areas.append(bin_sum)

        # Debugging statement for sum
        #print(f"Bin Intensity Sum = {bin_sum}")

        # Check if there are any intensities in the bin before proceeding
        if not bin_data['Intensity'].empty:
          if Calculate_Isotopes_and_Masses:
            max_y_index = bin_data['Intensity'].idxmax()
            corresponding_x_value = data['Mass'].iloc[max_y_index]  # Accessing using iloc
            isotopes.append(round(self.AMU_Element - (self.molecular_weight_of_molecule - corresponding_x_value)))
            masses.append(corresponding_x_value)
            #print(max_y_index)  # For debugging, Get the index of the max intensity
            #print(corresponding_x_value) # For debugging, see what x-value was found.
        else:
            print("Your data should not be missing values. Clean your data >:(") if self.errors else None
            break
        # Increments current_bin_start to move to the next bin
        current_bin_start += bin_size + 1 # Increment by the size of the bin

    # Store areas and isotopes in class attributes
    if file_number == 1:
        self.areas_1 = areas
        self.isotopes_1 = isotopes if Calculate_Isotopes_and_Masses else None
        self.masses_P = masses if Calculate_Isotopes_and_Masses else None
    else:
        self.areas_2 = areas
        self.isotopes_2 = isotopes if Calculate_Isotopes_and_Masses else None
        self.masses_D = masses if Calculate_Isotopes_and_Masses else None

    if self.print_information:
      print(f"Calculated Areas for file {file_number}: {areas}")
      print(f"Calculated Isotopes values for file {file_number}: {isotopes}") if Calculate_Isotopes_and_Masses else None
      print(f"Calculated Masses values for file {file_number}: {masses}") if Calculate_Isotopes_and_Masses else None
    return areas  # Returns the areas list, the isotope values are just stored.

    # For reference, the molar mass of Nd(NO3)4 is 392.2616 g/mol.

  def add_areas(self, row_start, row_end, bin_size, file_number, Calculate_Isotopes_and_Masses = False):
    "In some event where you have data files you need to add up but the peaks themselves are not in the same row index, you can use this function to use calculate_areas_and_isotopes in the exact same manner, except for different row start and row end"
    Areas = np.array(self.areas_1 if file_number == 1 else self.areas_2)
    self.calculate_areas_and_isotopes(row_start, row_end, bin_size, file_number, Calculate_Isotopes_and_Masses)
    New_areas = np.array(self.areas_1 if file_number == 1 else self.areas_2)
    Calculate_New_Areas = Areas + New_areas
    if file_number == 1:
      self.areas_1 = Calculate_New_Areas.tolist()
    else:
      self.areas_2 = Calculate_New_Areas.tolist()
    print(f"Added Areas: {Calculate_New_Areas.tolist()}") if self.print_information else None

    self.calculate_areas_and_isotopes(row_start, row_end, bin_size, file_number, Calculate_Isotopes_and_Masses)


  def calculate_ratio(self, Normalization_Number, *areas, file_number):
      """Calculates either the ratio of either a list of areas, several individual areas, or one individual area as inputted from *areas. The number will either assign the results to ratios_1 or ratios_2,
      Normalization_Number is an integer of the number you'll be using to normalize the data for the ratios as an integer value and file_number is also an integer value as to which file you want to calculate the ratios for."""
      if file_number != 1 and file_number != 2:
        print(f"An error will occur, you need to make the input 'number' either 1 or 2 to store the information in self.ratios_1 or self.ratios_2") if self.warnings else None
      else:
        ratio_lst = []
        if len(areas) == 1 and isinstance(areas[0], list):
            for area in areas[0]: #loops over the list inside areas[0] which is needed since the list in *args will become a tuple
              rat = area/Normalization_Number
              ratio_lst.append(rat) # Append each ratio between two peaks to the list
            if file_number == 1:
              self.ratios_1 = ratio_lst
            else:
              self.ratios_2 = ratio_lst
        else:
          for area in areas:
            rat = area / Normalization_Number
            ratio_lst.append(rat)
          if file_number == 1:
              self.ratios_1 = ratio_lst
          else:
              self.ratios_2 = ratio_lst
        print(f"Calculated Ratios for file {file_number}: {ratio_lst}") if self.print_information else None
        return ratio_lst

  """ Test cases for clarity as to how this function works:
    1.) For a list of areas: print(calculate_ratio(10, [20, 30, 40])) -> [2.0, 3.0, 4.0]
    2.) for Individual areas: print(calculate_ratio(10, 20, 30, 40)) -> [2.0, 3.0, 4.0]
    3.) For a single area: print(calculate_ratio(10, 20)) -> [2.0] which is the singular ratio"""

  def calculate_alpha(self, ratio_1, ratio_2):
      """Calculate the alpha of a specific ratio between the two files calculated ratios"""
      return ratio_2/ratio_1 if ratio_1 and ratio_2 is not None else None

  def make_alphas_list(self):
      if len(self.ratios_1) != len(self.ratios_2):
        print(f"Type Error: The amount of ratios from the two files are not equal, you should use calculate alpha to manually assign the peaks you want to compare") if self.warnings else None
      alphas_lst = []
      list_length_differences = max(len(self.ratios_1), len(self.ratios_2)) - min(len(self.ratios_1), len(self.ratios_2))
      for i in range(min(len(self.ratios_1), len(self.ratios_2))):
        current_alpha = self.ratios_2[i]/self.ratios_1[i]
        alphas_lst.append(current_alpha)
      if list_length_differences != 0:
        while list_length_differences != 0: #This ensures it won't error even with unequal list sizes.
          list_length_differences = list_length_differences - 1
          alphas_lst.append(None)
      self.alphas = alphas_lst
      print(f"Calculated Alphas: {alphas_lst}") if self.print_information else None
      return self.alphas

  def calculate_enrichment(self, alpha):
        return 1- alpha

  def enrichment_list(self):
      enrichment_list = [self.calculate_enrichment(alpha) for alpha in self.alphas]
      self.enrichments = enrichment_list
      print(f"Calculated Enrichment: {enrichment_list}") if self.print_information else None
      return enrichment_list

  def isotope_effect(self):
      YN_lst = self.YN_list
      for alpha in self.alphas:
        if alpha == 1:
          YN_lst.append('No')
        else:
          YN_lst.append('Yes')
      self.YN_list = YN_lst
      print(f"Calculated YN: {YN_lst}") if self.print_information else None
      return YN_lst

  def preferences(self):
      preference_lst = self.preference_list
      for enrichment in self.enrichments:
        if enrichment > 1:
          preference_lst.append("D") #preference for daughter species
        elif enrichment < 1:
          preference_lst.append("P") #preference for parent species
        else:
          preference_lst.append("=") #equal preference
      self.preference_list = preference_lst
      print(f"Calculated Preferences: {preference_lst}") if self.print_information else None
      return self.preference_list

  def drop(self, edit = False, *what_indexes, **what_list):
      """Drop specific values from the self.class attributes.

      Args:
          what_indexes (int): Indexes to drop from the list.
          edit (bool): If True, modifies the actual attribute by removing values.
          what_list (str): The name of the attribute to drop from.
      """
      for list_name, _ in what_list.items():
          if hasattr(self, list_name):
              target_list = getattr(self, list_name)
              if edit:
                  for i in sorted(what_indexes, reverse=True):  # Sort to avoid index shift
                      if i < len(target_list):
                          target_list.pop(i)
              else:
                  print([target_list[i] for i in what_indexes if i < len(target_list)]) if self.print_information else None
          else:
            print(f"Attribute '{list_name}' not found in the class.") if self.notices else None

# Error propagation
class Error_Propagation(Mass_Spectrometer_Data):
    def __init__(self, Mass_Spectrometer_Data_Instance):
        self.Data = Mass_Spectrometer_Data_Instance
        self.The_Method_Function_Access = The_Method()
        self.The_Method_Function_Access.copy_from(self.Data)
        self.parent_total_areas, self.daughter_total_areas = [], []
        self.Error_D, self.Error_P = [], []
        self.δ_PA, self.δ_DA = [],[]
        self.alphas = []
        self.enrichments = []
        self.δα = []
        self.Isotopes = []
        self.masses_P = []
        self.masses_D = []
        self.Parent_ratios = []
        self.Daughter_ratios = []
        self.print_information = False
        self.file_names_P = []
        self.file_names_D = []
        self.Preference_list = []

    def Individual_δ(self, *Areas):
      """Calculates the individual δ of every Area in the list or individual area depending what is passed in. This allows you to individually select Areas to find the error of."""
      if len(Areas) == 1 and isinstance(Areas[0], list):
        return np.sqrt(np.array(Areas[0])).tolist()
      else:
        return math.sqrt(Areas[0])

    def calculate_δα_indv(self, row_starts_and_ends_parent, row_starts_and_ends_daughter, Bin_size, number_of_peaks, normalization, parent_daughter, ROWSKIP = 7, NBTA = False, rows_are_masses = False):
      """Calculates δα for each individual area as a method of getting error bars. Then combines them. This method allows you to keep track of each individual error before the Areas are added up and keep track of all data being
      produced up to the calculation of the errors.

      Specifications:
      row_starts_and_ends_parent = [row_start_parent, row_end_parent] if you have evenly spaced peaks, otherwise you can put [[row_start_parent_1, row_end_parent_1], [row_start_parent_2, row_end_parent_2]] and so on for any unevenly spaced peaks.
      row_starts_and_ends_daughter = same as row_starts_and_ends_parent, but for daughters
      Bin_Size = integer of how many values you want for each rows in the spreedsheet
      number_of_peaks = integer of how many peaks corresponding to the lighter molecules you found.
      normalization = integer corresponding to number you want to use to normalize the data to.
      parent_daughter = = ['parent_NCE_Value', 'daughter_NCE_Value'] to compare between two NCE (of consistent isolations) data.
      NBTA is True if you want to normalize by the total areas, otherwise it'll use the normalization integer you put.

      """

      # So that the function knows these variables refer to the global variables when it is used in an if else statement to check that they're empty since they will be defined only once
      global All_Isotopes
      global Parent_masses
      global Daughter_masses
      global Atom_Names

      # Uses the specified NCE values as keys
      parent = parent_daughter[0]
      daughter = parent_daughter[1]

      # Gathers the file paths for the specified keys. Checks if the specified parent_daughter pair works, if not it'll attempt to use the file_paths that would be gathered from seperate_files.
      Parent_Files = self.Data.NCE_File_Paths[parent]
      Daughter_Files = self.Data.NCE_File_Paths[daughter]

      # Create all variables to store information in
      parent_areas, parent_error, daughter_areas, daughter_error, parent_percent_error, daughter_percent_error, δ_PA, δ_DA, δα= ([] for _ in range(9))
      xx, yy = np.zeros(number_of_peaks), np.zeros(number_of_peaks) # Total parent Areas array, Total daughter Areas array

      # To count through the file names for labeling purposes
      file_tracker_D = -1
      file_tracker_P = -1
      file_number_P, file_number_D = self.Data.numeric_file_indices[:len(Parent_Files)], self.Data.numeric_file_indices[len(Parent_Files):] #Actual number of file
      self.file_names_P = file_number_P
      self.file_names_D = file_number_D

      #Summing areas of the same peak areas across all specified files
      for file_P in Parent_Files:
        file_tracker_P += 1
        self.The_Method_Function_Access.raw_data_1 = pd.read_csv(file_P, skiprows = ROWSKIP)
        x = np.array([])
        for row_starts_and_ends in row_starts_and_ends_parent:
          start_P, end_P = row_starts_and_ends[0], row_starts_and_ends[1]
          if rows_are_masses:
            area_list = self.The_Method_Function_Access.calculate_areas_and_isotopes(start_P, end_P, Bin_size, 1, ROWSSKIPPED = ROWSKIP, rows_are_masses = True)
          else:
            area_list = self.The_Method_Function_Access.calculate_areas_and_isotopes(start_P - ROWSKIP, end_P - ROWSKIP, Bin_size, 1, ROWSSKIPPED = ROWSKIP)
          x = np.concatenate([x, area_list])
        print(f"Areas for P file {file_number_P[file_tracker_P]}: {x}") if self.print_information else None
        print(f" δ_PA file {file_number_P[file_tracker_P]}: {np.sqrt(x)}") if self.print_information else None
        xx = xx + x
      for file_D in Daughter_Files:
        file_tracker_D += 1
        self.The_Method_Function_Access.raw_data_2 = pd.read_csv(file_D, skiprows = ROWSKIP)
        y = np.array([])
        for row_starts_and_end in row_starts_and_ends_daughter:
            start, end = row_starts_and_end[0], row_starts_and_end[1]
            if rows_are_masses:
              area_lst = self.The_Method_Function_Access.calculate_areas_and_isotopes(start, end, Bin_size, 2, ROWSSKIPPED = ROWSKIP, rows_are_masses = True)
            else:
              area_lst = self.The_Method_Function_Access.calculate_areas_and_isotopes(start - ROWSKIP, end - ROWSKIP, Bin_size, 2, ROWSSKIPPED = ROWSKIP)
            y = np.concatenate([y, area_lst])
        print(f"Areas for D file {file_number_D[file_tracker_D]}: {y}") if self.print_information else None
        print(f" δ_DA file {file_number_D[file_tracker_D]}: {np.sqrt(y)}") if self.print_information else None
        yy = yy + y
      self.parent_total_areas = xx
      if self.Data.date_data_was_taken in All_Parent_Areas:
        key = self.Data.date_data_was_taken
        new_key = key
        # Looping to check if the key exists and to keep appending a prime until it's a unique key
        while new_key in All_Parent_Areas:
          new_key = new_key + "'"
        All_Parent_Areas[new_key] = tuple(xx)
      else:
        All_Parent_Areas[self.Data.date_data_was_taken] = tuple(xx)
      self.daughter_total_areas = yy
      if self.Data.date_data_was_taken in All_Daughter_Areas:
        key = self.Data.date_data_was_taken
        new_key = key
        # Looping to check if the key exists and to keep appending a prime until it's a unique key
        while new_key in All_Daughter_Areas:
          new_key = new_key + "'"
        All_Daughter_Areas[new_key] = tuple(yy)
      else:
        All_Daughter_Areas[self.Data.date_data_was_taken] = tuple(yy)
      print(f"total areas for P: {xx}") if self.print_information else None
      print(f"total areas for D: {yy}") if self.print_information else None
      δ_PA = np.sqrt(xx)
      δ_DA = np.sqrt(yy)
      self.δ_PA = δ_PA
      self.δ_DA = δ_DA
      print(f"δ_PA: {δ_PA}") if self.print_information else None
      print(f"δ_DA: {δ_DA}") if self.print_information else None
      Error_P = δ_PA/xx
      Error_D= δ_DA/yy
      print(f"% error for P: {Error_P}") if self.print_information else None
      print(f"% error for D: {Error_D}") if self.print_information else None
      self.Error_D = Error_D
      self.Error_P = Error_P
      daughter_ratios = yy/np.sum(yy) if NBTA else yy/normalization
      parent_ratios = xx/np.sum(xx) if NBTA else xx/normalization
      print(f"Parent Ratios: {parent_ratios}") if self.print_information else None
      print(f"Daughter Ratios: {daughter_ratios}") if self.print_information else None
      self.Parent_ratios = parent_ratios
      self.Daughter_ratios = daughter_ratios

      alpha = daughter_ratios/parent_ratios
      self.alphas = alpha
      print(f"Alpha Values: {alpha}") if self.print_information else None
      if self.Data.date_data_was_taken in All_alphas:
        key = self.Data.date_data_was_taken
        new_key = key
        while new_key in All_alphas:
          new_key = new_key + "'"
        All_alphas[new_key] = tuple(alpha)
      else:
        All_alphas[self.Data.date_data_was_taken] = tuple(alpha) # Puts the alpha values in the global variable that is a dictionary of all alphas corresponding to a key which is the date the data was taken.

      Enrichment = alpha - 1
      self.enrichments = Enrichment
      print(f"Enrichment Values: {Enrichment}") if self.print_information else None
      if self.Data.date_data_was_taken in All_enrichments:
        key = self.Data.date_data_was_taken
        new_key = key
        while new_key in All_enrichments:
          new_key = new_key + "'"
        All_enrichments[new_key] = tuple(Enrichment)
      else:
        All_enrichments[self.Data.date_data_was_taken] = tuple(Enrichment) # Puts the alpha values in the global variable that is a dictionary of all alphas corresponding to a key which is the date the data was taken.

      δα = (alpha * np.sqrt((Error_D) ** 2 + (Error_P) ** 2))
      self.δα = δα
      print(f"δα: {δα}") if self.print_information else None
      if self.Data.date_data_was_taken in All_Errors:
        key = self.Data.date_data_was_taken
        new_key = key
        while new_key in All_Errors:
          new_key = new_key + "'"
        All_Errors[new_key] = tuple(δα)
      else:
        All_Errors[self.Data.date_data_was_taken] = tuple(δα) # Puts the Error values in the global variable that is a dictionary of all Errors corresponding to a key which is the date the data was taken.

      #Getting the isotopes and masses
      self.The_Method_Function_Access.raw_data_1 = pd.read_csv(Parent_Files[0], skiprows = ROWSKIP)
      ISOTOPES = []
      PARENT_MASSES = []
      DAUGHTER_MASSES = []
      self.The_Method_Function_Access.raw_data_1 = pd.read_csv(Parent_Files[0], skiprows = ROWSKIP)
      self.The_Method_Function_Access.raw_data_2 = pd.read_csv(Daughter_Files[0], skiprows = ROWSKIP)
      for row_starts_and_ends in row_starts_and_ends_parent:
        start_P, end_P = row_starts_and_ends[0], row_starts_and_ends[1]
        if rows_are_masses:
          self.The_Method_Function_Access.calculate_areas_and_isotopes(start_P, end_P, Bin_size, 1, ROWSSKIPPED = ROWSKIP, Calculate_Isotopes_and_Masses = True, rows_are_masses = True)
        else:
          self.The_Method_Function_Access.calculate_areas_and_isotopes(start_P - ROWSKIP, end_P - ROWSKIP, Bin_size, 1, ROWSSKIPPED = ROWSKIP, Calculate_Isotopes_and_Masses = True)
        for i in self.The_Method_Function_Access.isotopes_1:
          ISOTOPES.append(i)
        for j in self.The_Method_Function_Access.masses_P:
          PARENT_MASSES.append(j)
      print(f"Isotopes: {ISOTOPES}") if self.print_information else None
      if All_Isotopes == (): # This is to make the All_Isotopes global variable have the isotopes. This only occurs once when the tuple is empty.
        All_Isotopes = tuple(ISOTOPES)
      print(f"Masses Parent: {PARENT_MASSES}") if self.print_information else None
      for row_starts_and_ends in row_starts_and_ends_daughter:
        start_D, end_D = row_starts_and_ends[0], row_starts_and_ends[1]
        if rows_are_masses:
          self.The_Method_Function_Access.calculate_areas_and_isotopes(start_D, end_D, Bin_size, 2, ROWSSKIPPED = ROWSKIP, Calculate_Isotopes_and_Masses= True, rows_are_masses = True)
        else:
          self.The_Method_Function_Access.calculate_areas_and_isotopes(start_D - ROWSKIP, end_D - ROWSKIP, Bin_size, 2, ROWSSKIPPED = ROWSKIP, Calculate_Isotopes_and_Masses= True)
        for k in self.The_Method_Function_Access.masses_D:
          DAUGHTER_MASSES.append(k)
      print(f"Masses Daughter: {DAUGHTER_MASSES}") if self.print_information else None
      self.Isotopes = ISOTOPES
      self.masses_P = PARENT_MASSES
      self.masses_D = DAUGHTER_MASSES
      #For storing parent and daughter masses in the Parent_masses and Daughter_Masses global variables only once when they are empty
      if Parent_masses == ():
        Parent_Masses = tuple(PARENT_MASSES)
      if Daughter_masses == ():
        Daughter_Masses = tuple(DAUGHTER_MASSES)

      if Atom_Names == " ":
        Atom_Names = self.The_Method_Function_Access.atom_name

      self.Preference_list = np.array(['Parent' if ε < 0 else 'Neither' if ε == 0 else 'Daughter' for ε in self.enrichments])
      # If you notice any data you want to remove, use the drop_files function in the parent class Mass_Spectrometer_Data, run the get_file_paths function again, then perform all calculations

# Visualizing the data
class Graphs():
  """Used to generate all types of Graphs for the data between two specific values after all the data has been produced in the other Classes. There are graphs that are in the global function section.
  The save_pdf in each graph is used to not do plt.show() so that it can be saved as a figure instead in the Make_PDF class (True to make_pdfs, False to plot the graph)."""
  def __init__(self, Mass_Spectrometer_Data_Instance, Error_instance):
    self.MSDI = Mass_Spectrometer_Data_Instance #Specify for which graphs you want to create.
    self.error = Error_instance  #Specify Errors for which graphs you'd want to create
    self.professional = False

  def IVM(self, file_path, xlim_left, xlim_right, plot_title, save_pdf = False):
    """Generates a Intensity Vs Mass graph from the data
    Enter how many rows you need to skip to access the data into row_skip as an integer
    Enter where you want the graphs to start and end to zoom in as integers, into xlim_left and xlim_right respectively
    Enter the plot_title as a string of what you want to name the graph into plot_title"""
    data = pd.read_csv(file_path, skiprows = 7)
    Column_labels = ['Mass', 'Intensity']
    data.columns = Column_labels
    x_axis = 'Mass'
    y_axis = 'Intensity'
    plt.plot(data[x_axis], data[y_axis], label = f'{y_axis} vs {x_axis}')
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    plt.title(plot_title)
    plt.xlim(xlim_left, xlim_right)
    if not save_pdf:
       plt.show()

  def generate_all_IVM(self, xlim_left_P, xlim_right_P, xlim_left_D, xlim_right_D, Parent_Daughter, save_pdf = False):
      """Generates all Intensity versus Masses graphs for every file for the parent and daughter species. I assume you only have two for this"""
      Parent_key = Parent_Daughter[0] #To get the file_paths for the specifc NCE values in NCE_File_Paths
      Daughter_key = Parent_Daughter[1]
      P = 0 #Initalize counters to keep number the files in order
      D = 0
      for i in self.MSDI.NCE_File_Paths[Parent_key]:
        plot_title_P = "Intensity Vs Mass" + " " + "For Parent File:" + " " + str(P) + " " + "on" + " " + self.MSDI.date_data_was_taken
        self.IVM(i, xlim_left_P, xlim_right_P, plot_title_P) if not save_pdf else self.IVM(i, xlim_left_P, xlim_right_P, plot_title_P, save_pdf = True)
        P += 1
      for j in self.MSDI.NCE_File_Paths[Daughter_key]:
        plot_title_D = "Intensity Vs Mass" + " " + "for Daughter File:" + " " + str(D) + " " + "on" + " " + self.MSDI.date_data_was_taken
        self.IVM(j, xlim_left_D, xlim_right_D, plot_title_D) if not save_pdf else self.IVM(j, xlim_left_D, xlim_right_D, plot_title_D, save_pdf = True)
        D += 1

  def Big_IVM(self, Parent_Daughter, save_pdf = False):
      """IVM showing the total combined data frames from all the files for each NCE Parent_Daughter is list of strings for NCE values you are representing as parent and daughter such as ['0','70']"""
      Parent_key = Parent_Daughter[0]
      Daughter_key = Parent_Daughter[1]
      Parent =  self.MSDI.NCE_dataframes[Parent_key]
      Daughter = self.MSDI.NCE_dataframes[Daughter_key]
      Column_labels = ['Mass', 'Intensity']
      Parent.columns = Column_labels
      Daughter.columns = Column_labels
      x_axis = 'Mass'
      y_axis = 'Intensity'
      plt.plot(Parent[x_axis], Parent[y_axis], label = f'{y_axis} vs {x_axis}')
      plt.xlabel(x_axis)
      plt.ylabel(y_axis)
      plt.title("Fragmentation Analysis: Intensity Patterns Across Parent Molecule Masses" + self.MSDI.date_data_was_taken)
      plt.show()
      plt.plot(Daughter[x_axis], Daughter[y_axis], label = f'{y_axis} vs {x_axis}')
      plt.xlabel(x_axis)
      plt.ylabel(y_axis)
      plot_title_D = "Fragmentation Analysis: Intensity Patterns Across Daughter Molecule Masses" + self.MSDI.date_data_was_taken
      plt.title(plot_title_D)
      if not save_pdf:
       plt.show()

  def εVIs(self, save_pdf = False):
    """Generates an Basic Enrichment vs Isotope Graph with error bars from data calculated in a method. Make the plot title a string of the title you want."""
    Column_labels = ['Isotope', 'ε']
    x_axis = self.error.Isotopes
    y_axis = self.error.enrichments
    yerr = self.error.δα
    custom_labels = []
    for isotope in self.error.Isotopes:
      isotope_label = f"$\\mathrm{{^{{{isotope}}}{self.MSDI.atom_name}}}$"
      custom_labels.append(isotope_label)  # Custom labels
    plt.plot(x_axis, y_axis, color = '#1a2b6d') if self.professional else plt.plot(x_axis, y_axis, color = (0.6, 0.8, 1.0))
    plt.xticks(x_axis, custom_labels)
    plt.xlabel('Isotope')
    plt.ylabel('ε')
    plt.title(("Exploring the Relationship Between Enrichment Levels and Isotopic Composition" + " " + self.MSDI.date_data_was_taken))
    plt.errorbar(x_axis, y_axis, yerr=yerr, fmt='s', color ='#8b0000', ecolor= '#888888', capsize=15) if self.professional else plt.errorbar(x_axis, y_axis, yerr=yerr, fmt='s', color = (1.0,0.5,0.4), ecolor= (0.5,0.0,0.5), capsize=15) # A soft serenity color theme
    graph_name = "Exploring the Relationship Between Enrichment Levels and Isotopic Composition" + " " + self.MSDI.date_data_was_taken
    for i, txt in enumerate(self.error.enrichments.tolist()):
        plt.annotate(txt, (x_axis[i], y_axis[i]))
    if not save_pdf:
       plt.show()

  def αVIs(self, save_pdf = False):
    """Generates a Alpha Vs Isotope graph from the data"""
    # Purely for Decoration
    if not self.professional:
      Color_Themes_Picker = np.array([0,1,2,3])
      Random_Theme = np.random.choice(Color_Themes_Picker)
      Line_Colors = [(0.0,0.2,0.8), '#6dbb22', '#d52d00', '#3b9d1e' ]
      Dot_Colors = [(0.7,0.5,0.9), '#8c5c2f', '#0033cc', '#2e8b57' ]
      Error_Bar_colors = ['green',  '#4a7023', '#003399', '#001f4d' ]
    Column_labels = ['Isotope', 'α']
    x_axis = self.error.Isotopes
    custom_labels = []
    for isotope in self.error.Isotopes:
      isotope_label = f"$\\mathrm{{^{{{isotope}}}{self.MSDI.atom_name}}}$"
      custom_labels.append(isotope_label)  # Custom labels
    y_axis = self.error.alphas
    yerr = self.error.δα
    plt.plot(x_axis, y_axis, color = '#1a2b6d') if self.professional else plt.plot(x_axis, y_axis, color = Line_Colors[Random_Theme])
    plt.xticks(x_axis, custom_labels)
    plt.xlabel('Isotope')
    plt.ylabel('α')
    plt.title("Alpha Coefficient Trends as a Function of Isotopic Composition" + " " + self.MSDI.date_data_was_taken)
    plt.errorbar(x_axis, y_axis, yerr=yerr, fmt='s', color ='#8b0000', ecolor= '#888888', capsize=15) if self.professional else plt.errorbar(x_axis, y_axis, yerr=yerr, fmt='s', color = Dot_Colors[Random_Theme], ecolor = Error_Bar_colors[Random_Theme], capsize=15)
    graph_name = "Alpha Coefficient Trends as a Function of Isotopic Composition" + " " + self.MSDI.date_data_was_taken
    for i, txt in enumerate(self.error.alphas.tolist()):
        plt.annotate(txt, (x_axis[i], y_axis[i]))
    if not save_pdf:
       plt.show()


  def RIVM(self, PDF_Maker = None):
    """Relative intensity vs masses for parent and daughter graphs"""
    # Purely for Decoration
    if not self.professional:
      Color_Themes_Picker_1 = np.array([0,1,2,3,4,5,6,7])
      Color_Themes_Picker_2 = np.array([0,1,2,3,4,5,6,7])
      Random_Theme_1 = np.random.choice(Color_Themes_Picker_1)
      Random_Theme_2 = np.random.choice(Color_Themes_Picker_2)
      Line_Colors_1 = [(0.7,0.1,0.1), '#a23b3b', '#d16002', '#ff6f61', '#FFD700', "#8C1515", "#FFA500", "#D4A017"]
      Dot_Colors_1 = [(1.0, 0.85, 0.0), '#d9a673', '#9e2a2b','#f28a30', '#FF6F61', "#B3995D","#FF69B4", "#3366CC"]
      Error_Bar_colors_1 = [(1.0, 0.6, 0.2), '#ffb087', '#b17a34', '#f7c6a3', '#FFB6A0', "#4D4F53","#FFD700", "#FFCC33"]
      Line_Colors_2 = [(0.2,0.4,0.2),  '#2a4d69', '#006d77', '#191970', '#2e5d55', '#0f4c81', "#003262", "#0099FF", "#87CEEB"]
      Dot_Colors_2 = [(0.5,0.25,0.1),  '#81894e', '#ff6b6b', '#7a7bb0', '#4682b4', '#84c0c6', "#FDB515", "#CC6600", "#8B4513"]
      Error_Bar_colors_2 = [(0.6,1.0,0.6), '#4b8e8d', '#83c5be', '#a4c3e2', '#b0c4de', '#d8e2dc', "#BDC3C7", "#66BB66", "#228B22"]
    #Actual plotting
    plt.figure(figsize = (10,5))
    Column_labels = ['Mass', 'Relative Intensity']
    x_axis = self.error.masses_P #rounded
    y_axis = self.error.parent_total_areas.tolist()
    yerr = self.error.Error_P
    plt.plot(x_axis, y_axis, color = '#1a2b6d') if self.professional else plt.plot(x_axis, y_axis, color = Line_Colors_1[Random_Theme_1])
    ratio_label_P = self.error.Parent_ratios.tolist()
    plt.xlabel('Mass')
    plt.ylabel('Relative Intensity')
    plt.title("Relative Intensity by Mass in Parent Molecule Fragmentation" + " " + self.MSDI.date_data_was_taken)
    plt.errorbar(x_axis, y_axis, yerr=yerr, fmt='s', color ='#8b0000', ecolor= '#888888', capsize=15) if self.professional else plt.errorbar(x_axis, y_axis, yerr=yerr, fmt='s', color = Dot_Colors_1[Random_Theme_1], ecolor = Error_Bar_colors_1[Random_Theme_1] , capsize=15)
    graph_name = "Relative Intensity by Mass in Parent Molecule Fragmentation" + " " + self.MSDI.date_data_was_taken
    for i, txt in enumerate(self.error.Parent_ratios):
        plt.annotate(txt, (x_axis[i], y_axis[i]))
    if PDF_Maker is not None:
      PDF_Maker.pdf_pages.savefig(dpi=300)
      plt.close()
    else:
      plt.show()

    plt.figure(figsize = (10,5))
    Column_labels = ['Mass', 'Relative Intensity']
    x_axis = self.error.masses_D # rounded
    y_axis = self.error.daughter_total_areas.tolist()
    yerr = self.error.Error_D
    plt.plot(x_axis, y_axis, color = '#1a2b6d') if self.professional else plt.plot(x_axis, y_axis, color = Line_Colors_2[Random_Theme_2])
    plt.errorbar(x_axis, y_axis, yerr=yerr, fmt='s', color ='#8b0000', ecolor= '#888888', capsize=15) if self.professional else plt.errorbar(x_axis, y_axis, yerr=yerr, fmt='s', color = Dot_Colors_2[Random_Theme_2], ecolor = Error_Bar_colors_2[Random_Theme_2], capsize=15)
    plt.xlabel('Mass')
    plt.ylabel('Relative intensity')
    plt.title("Relative Intensity by Mass in Daughter Molecule Fragmentation" + " " + self.MSDI.date_data_was_taken)
    graph_name = "Relative Intensity by Mass in Daughter Molecule Fragmentation" + " " + self.MSDI.date_data_was_taken
    for i, txt in enumerate(self.error.Daughter_ratios):
        plt.annotate(txt, (x_axis[i], y_axis[i]))
    if PDF_Maker is not None:
      PDF_Maker.pdf_pages.savefig(dpi=300)
      plt.close()
    else:
      plt.show()

class MakePDF:
    """ Example Use of MakePDF:

    Use the MakePDF class with a 'with' statement:
    with MakePDF("example_with_context.pdf", "path/to/save") as pdf_creator:
    pdf_creator.add_chart(generate_line_chart, data)
    Add a table to the PDF:
    pdf_creator.add_table(table_data, column_names=column_names, additional_rows=additional_rows)

    Alternatively, you can use each function individually if the table requires some changes that you don't want to manually type out
    """
    def __init__(self, pdf_filename, file_path=None):
        """
        Initializes the PDF generator.

        Args:
            pdf_filename (str): The name of the PDF file where content will be saved.
        """
        # Handle file path; use pdf_filename if no path provided
        if not pdf_filename.endswith('.pdf'):
            pdf_filename += '.pdf'
        self.pdf_path = pdf_filename if not file_path else file_path + '/' + pdf_filename
        self.pdf_pages = PdfPages(self.pdf_path)
        self.table_data = None # Holds the table itself in it's list of list form.
        self.debug = False # Set to True if you want to have debug statements on.
        self.warnings = False
        self.Errors = False
        self.notice = False

    def pivot_table_data(self, *args):
        """
        Pivots columns of lists of data into rows for the table.

        Args:
            *args: Multiple lists of data to be arranged vertically into table rows.

        Returns:
            list: A list of rows (lists), each containing values from the input lists.
        """
        # Combine the data into rows (transpose)
        table_data = list(zip(*args))  # This effectively transposes the data
        return table_data

    def handle_floats_and_transpose(self, data_sets, Number_of_decimals = None):
      float_counter = 1
      placeholder_map = {}  # to store float->placeholder mappings
      transformed_data_sets = []

      # Step 1: Replace float values with unique codes
      for data in data_sets:
          print(f"Processing data set: {data}") if self.debug else None # Check each data set before processing
          transformed_data = []

          # Ensure all rows have the same length
          row_lengths = [len(row) for row in data]
          if len(set(row_lengths)) > 1:
              print(f"Warning: Rows have unequal lengths in data set. Skipping transposition for inconsistent data.") if self.warnings else None
              continue  # Or handle the rows in some other way

          for row in data:
              print(f"Processing row: {row}") if self.debug else None  # Check each row
              transformed_row = []
              for value in row:
                  print(f"Processing value: {value} (type: {type(value)})") if self.debug else None # Check the value
                  if isinstance(value, np.float64):
                      # Assign a unique code to the floating point value
                      placeholder = f'FLOAT{float_counter}'
                      transformed_row.append(placeholder)
                      placeholder_map[placeholder] = value  # map the placeholder to the actual value
                      float_counter += 1
                  else:
                      transformed_row.append(value)
              transformed_data.append(transformed_row)
          transformed_data_sets.append(transformed_data)

      # Step 2: Transpose the data (pivot)
      print(f"Transposing data sets...") if self.debug else None
      pivoted_data_sets = [list(zip(*data)) for data in transformed_data_sets]

      # Step 3: Restore the floats from the placeholders
      restored_data_sets = []
      for data in pivoted_data_sets:
          restored_data = []
          for row in data:
              restored_row = []
              for value in row:
                  if isinstance(value, str) and value.startswith('FLOAT'):
                      # Replace the placeholder with the original value
                      original_value = placeholder_map.get(value, value)
                      if Number_of_decimals is not None:
                        # Round to the specified number of decimals
                        original_value = round(original_value, Number_of_decimals)
                      restored_row.append(original_value)
                      """# Replace the placeholder with the original value
                      restored_row.append(placeholder_map.get(value, value))"""
                  else:
                      restored_row.append(value)
              restored_data.append(restored_row)
          restored_data_sets.append(restored_data)

      return restored_data_sets

    def define_table(self, column_names, row_names, *data_sets, pivot=False, number_of_decimals = None):
      """
      Defines the table with the specified row names, column headers, and multiple data sets.
      Optionally pivots each data set (switches rows and columns).

      Args:
          column_names (list): List of column names (e.g., ['File 0', 'File 1', 'File 2']).
          row_names (list): List of row names (e.g., ['Alpha', 'Enrichment', 'Error']).
          *data_sets (lists): Multiple lists of data (e.g., [enrichment_data], [alpha_data], [ratio_data]).
          pivot (bool): If True, pivots the data sets (rows become columns and columns become rows).
          number_of_decimals (int): Number of decimal places to include when displaying data. It will round the last digit to the number of decimals specified.
      Returns:
          list: A 2D list representing the table with row names, column headers, and data.
      """
      # Check if we need to pivot
      # Step 1: Handle floaters and transpose the data
      if pivot:
            # Use handle_floats_and_transpose to replace floats, transpose, and restore values
            data_sets = self.handle_floats_and_transpose(data_sets, Number_of_decimals = number_of_decimals)
      else:
        # If not pivoting, handle rounding directly if it is specified
        if number_of_decimals is not None:
            rounded_data_sets = []
            for data in data_sets:
                rounded_data = [
                    [round(value, number_of_decimals) if isinstance(value, (float, np.float64)) else value for value in row]
                    for row in data
                ]
                rounded_data_sets.append(rounded_data)
            data_sets = rounded_data_sets

      # Step 2: Create the final table with an empty cell in the first column and column headers
      table_data = []

      # Add the column headers to the first row (empty cell for 'File')
      table_data.append([''] + row_names.tolist())

      # Add each row (file) from the pivoted data
      for i, file_name in enumerate(column_names):
            row = [file_name]  # Add the file name in the first column
            for data in data_sets:
                row.extend(data[i])  # Add data corresponding to each file
            table_data.append(row)

      self.table_data = table_data
      return table_data

    def truncate_text(cell_text, max_char):
        """Truncates text to fit within a specified character limit per cell."""
        if isinstance(cell_text, str) and len(cell_text) > max_char:
            return cell_text[:max_char] + "..."  # Truncate and add ellipsis
        return cell_text  # Return the text if it's short enough

    def add_table(self, table_data, title = 'I owe you a title', Additional_Information = None):
        """
        Adds a table to the PDF.
        Args:
            table_data (list): A 2D list representing the table data. You can pass in define_table() here
            title (str): A title for the table.
            Additional_Information (str): Additional information to be displayed below the table. Such as Isolations and NCE values.
        """
        # Create table plot
        ax = plt.gca()
        ax.axis('tight')
        ax.axis('off')

        # Create the table
        table = plt.table(
            cellText=table_data,
            loc='center',
            colWidths=[0.2] * (len(table_data[0]) if table_data else 1),
            cellLoc='center'
        )

        # Adjust table appearance
        table.auto_set_font_size(False)
        table.set_fontsize(12)  # Set a readable font size
        table.scale(1.5, 1.5)   # Scale table width and height for better visibility

        # Add a title if provided
        if title:
            plt.title(title, fontsize=14, weight='bold', pad=20)  # Add spacing from the table

        table.auto_set_font_size(False)
        table.set_fontsize(10)  # Adjust font size to fit content better

        if Additional_Information is not None:
          # Add additional information below the table
          ax.text(0.5, -0.5, Additional_Information,
          fontsize=10, ha='center', transform=ax.transAxes)

        #Alternative method to fit data in the cell if it is preferable
        """ #Apply truncation to all cells in the table data
        truncated_table_data = [
            [truncate_text(str(cell), max_char) for cell in row]
            for row in table_data
        ]"""

        # Save the table to the PDF
        self.pdf_pages.savefig(bbox_inches='tight', dpi=300)
        plt.close()

    def add_chart(self, chart_function, **additional_arguments):
        """
        Adds a chart to the PDF using an existing chart function.

        Args:
            chart_function (function): A pre-existing function that generates a chart (e.g., `overlaid_line_plot_AVI,`).
            **additional_arguments: Additional arguments to pass to the chart function.
        """
        # Generate the chart using the provided chart function
        plt.figure(figsize = (10,5))
        chart_function()
        # Save the chart to the PDF
        chart_function(**additional_arguments)  # Pass kwargs to the chart function
        self.pdf_pages.savefig(dpi=300)
        plt.close()

    def save_pdf(self):
        """Closes the PDF file and saves all pages."""
        self.pdf_pages.close()

    # If you want to use a with statement, these functions are implemented for this reason
    def __enter__(self):
        """Start the context and open the PDF for writing."""
        self.pdf_pages = PdfPages(self.pdf_path)
        return self  # Return the instance for use within the 'with' block

    def __exit__(self, exc_type, exc_val, exc_tb):
        """End the context and close the PDF."""
        if self.pdf_pages:
            self.pdf_pages.close()

